# Descriptive Analysis and Visualisations of the data

## Structure of cleaned file

- 'ID': Row ID
- 'Patient_ID': Unique patient identifier
- 'Age': of individual in whole numbers; 
    - (-5) = missing value
- 'Sex': of individual;
    - "Not reported" = missing value
- 'Date_Collected': date sample was collected
- 'Time_collected': time sample was collected
- 'DateTime_Collected': combination of date and time collected columns
- 'Result_1': test result
    - YES = detected
    - NO = not detected
- 'Facility_Code': Facility where the test was ordered
- 'AgeGroup' of individual:
    - 'Not reported',
    - '0 to 16',
    - '17 to 64',  
    - '65 and above'
- 'PostCode': Facility postcode
- 'Area': Facility remoteness as defined by [Queensland Government Statistician’s Office Place Name Concordance data 2019](https://www.qgso.qld.gov.au/geographies-maps/concordances/place-names-concordance-2019)
    - Outer Regional Australia
    - Very Remote Australia
    - Remote Australia
    - Inner Regional Australia
    - Major Cities of Australia'
- 'DaySinceFirstTest': Number of days since the first test; 
    - where 0 = first test
- 'DaySinceFirstPOSITIVETest':Number of days since the first POSITIVE test;
    - Negative numbers = negative tests that were conducted prior to first positive result
    - 0 = day of first positive result
    - Positive numbers = days since first positive result      
- 'NumberOfTests': Total number of tests conducted
- 'NumberOfTestsSinceFirstPOSITIVE': Number of tests *since and including* first positive
- 'NumberOfTestsBeforeFirstPOSITIVE': Number of tests *before but NOT including* first positive
- 'NumberOfPosTests': Total number of tests that came back positive
- 'NumberOfNegTests': Total number of tests that came back negative
- 'MaxDayTestSinceFirstPOSITIVETest': maximum number of days since first positive test
- 'MaxDayTestBEFOREFirstPOSITIVETest': maximum number of days before first positive test
- 'Status': clinical status of individual:
    - POS = positive - only need ONE pos result for pos status
    - NEG = negative - need TWO consecutive neg results $\geq$ 24 hours apart
- 'Status_Change': change from a negative to a positive after negative status established for a previously positive individual
    - 1 = status change occured from POS to NEG
    - 2 = status change occured from NEG to POS
    - Nan/null = no status change
- 'DaysToNegStatus': Number of days until the **first day of clinical negative status**; used in the primary analysis
- 'DaysToNegStatus_sensitiv':  Number of days until the last day of data; used in the sensitvity analysis
- 'DaysToNegStatus_biol': Number of days until **first day of biological negative status**
- 'DaysToNegStatus_sensitiv_biol': 
    - IF final status is positive = Number of days until last day of data
    - IF final status is negative = Number of days biological negative status is achieved; ie day of first of the two final negative results.    
- DaysToNegStatus_sensitivCensored
    - Days to first negative status were negative status is either of the following:
        1. Two neg tests: neg status will be counted as being achieved on the first of these two tests; OR
        2. A final negative tests were no additional data is provided
- DaysToNegStatus_sensitivUNCensored
    - As above; however, individuals are not censored after the first negative status
- 'AreaFirstTest': remoteness of first test
- 'PostCodeFirstTest': postcode of first test
- 'FacilityFirstTest': facility name of first test 


## Structure of Population - Area file

- 'PostCode': Queensland postcodes
- 'Area': remoteness (as above)
- 'Tota Population': population of postcode (2016 Census data) 


## Structure of Lab Number - Area file

- 'Lab_No': unique tests (Lab number) in original dataset prior to removal of multiple tests per-day
- 'Area': remoteness (as above)


# Load Libraries

In [ ]:
#VISUALS and WRANGLING
library(lubridate)


library(dplyr)
library(plyr)



library(ggplot2)
library(reshape2)
library(RColorBrewer)
library(egg)

library(gridExtra)
library(grid)

#discriptive analysis
library(DescTools)
library(epitools)
library(PMCMRplus)

#TIME TO EVENT ANALYSIS

library(ranger)
#library(ggfortify)
library(survival)

library(ggpubr)
library(survminer)




In [ ]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 17134)

Matrix products: default

locale:
[1] LC_COLLATE=English_Australia.1252  LC_CTYPE=English_Australia.1252   
[3] LC_MONETARY=English_Australia.1252 LC_NUMERIC=C                      
[5] LC_TIME=English_Australia.1252    

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] survminer_0.4.7    ggpubr_0.4.0       survival_3.2-3     ranger_0.12.1     
 [5] epitools_0.5-10.1  DescTools_0.99.36  egg_0.4.5          gridExtra_2.3     
 [9] RColorBrewer_1.1-2 reshape2_1.4.3     ggplot2_3.1.1      plyr_1.8.4        
[13] dplyr_1.0.0        lubridate_1.7.4   

loaded via a namespace (and not attached):
 [1] tidyr_1.1.0       jsonlite_1.6      splines_3.6.3     carData_3.0-4    
 [5] expm_0.999-4      cellranger_1.1.0  pillar_1.4.4      backports_1.1.4  
 [9] lattice_0.20-38   glue_1.4.1        uuid_0.1-2        digest_0.6.18    
[13] ggsignif_0.6.0    colorspace_1.4-1  htmltools_0.3.6   Matrix_1.2-17    
[17] pkgconfig_2.0.2   broom_0.5.6       haven_2.1.0       purrr_0.3.2      
[21] xtable_1.8-4      mvtnorm_1.1-1     scales_1.0.0      km.ci_0.5-2      
[25] openxlsx_4.1.5    rio_0.5.16        KMsurv_0.1-5      tibble_3.0.1     
[29] generics_0.0.2    car_3.0-8         ellipsis_0.3.1    withr_2.1.2      
[33] repr_0.19.2       lazyeval_0.2.2    magrittr_1.5      crayon_1.3.4     
[37] readxl_1.3.1      evaluate_0.13     nlme_3.1-139      MASS_7.3-51.3    
[41] rstatix_0.6.0     forcats_0.4.0     foreign_0.8-71    tools_3.6.3      
[45] data.table_1.12.2 hms_0.4.2         lifecycle_0.2.0   stringr_1.4.0    
[49] munsell_0.5.0     zip_2.0.4         compiler_3.6.3    rlang_0.4.6      
[53] pbdZMQ_0.3-3      IRkernel_0.8.15   rstudioapi_0.11   base64enc_0.1-3  
[57] boot_1.3-20       gtable_0.3.0      abind_1.4-5       curl_3.3         
[61] R6_2.4.0          zoo_1.8-5         knitr_1.22        survMisc_0.5.5   
[65] stringi_1.4.3     IRdisplay_0.7.0   Rcpp_1.0.1        vctrs_0.3.1      
[69] tidyselect_1.1.0  xfun_0.6         



# Load Data

In [ ]:
SwabData <- read.csv('Outputs/WrangledData/all_results.csv', header=TRUE)

rownames(SwabData) <- SwabData$ID

#remove the first two columns they're not needed
SwabData <- SwabData[, -c(1, 1)]

head(SwabData)

"***********************"


PopulationArea <- read.csv("Outputs/WrangledData/PopulationPostcodeArea.csv", header=TRUE)
rownames(PopulationArea) <- PopulationArea$X
PopulationArea <- PopulationArea[, -c(1, 1)]

head(PopulationArea)

"***********************"

Lab_NoArea <- read.csv("Outputs/WrangledData/tests_area.csv", header=TRUE)

rownames(Lab_NoArea) <- Lab_NoArea$X
Lab_NoArea <- Lab_NoArea[, -c(1, 1)]

head(Lab_NoArea)




"***********************"

all_data_preCalcs <- read.csv("Outputs/WrangledData/all_data_preCalcs.csv", header=TRUE)

rownames(all_data_preCalcs) <- Lab_NoArea$X
all_data_preCalcs <- all_data_preCalcs[, -c(1, 1)]

head(all_data_preCalcs)





In [ ]:
#give numberical values to STATUS positive = 1 and negative = 0 in new column 
SwabData$Status_Num[SwabData$Status == "POS"] <- 0 

SwabData$Status_Num[SwabData$Status == "NEG"] <- 1 

head(SwabData)

In [ ]:
str(SwabData)

In [ ]:
"Total number of tests"
length(unique(Lab_NoArea$Lab_No))

#min(unique(SwabData$Date_Collected))

# Create Additional Cohorts

## 1. All data: SwabData

No wrangling required

### 1.1 All data patient info: SwabData_PATIENTS

In [ ]:
#creates a df of unique patients with age, and sex 

SwabData_PATIENTS <- unique(select(SwabData, 'Patient_ID',
                                                                 'Age',
                                                                 'Sex',
                                                                 #'Date_Collected',
                                                                 #'Time_collected',
                                                                 #'DateTime_Collected',
                                                                 #'Result_1', 
                                                                 #'Facility_Description',
                                                                 'AgeGroup',
                                                                 #'PostCode',
                                                                 #'Area',
                                                                 #'Total.Population',
                                                                 #'DaySinceFirstTest',
                                                                 #'DaySinceFirstPOSITIVETest',
                                                                 'NumberOfTests',
                                                                 'NumberOfTestsSinceFirstPOSITIVE',
                                                                 'NumberOfTestsBeforeFirstPOSITIVE',
                                                                 'NumberOfNegTests',
                                                                 'NumberOfPosTests',
                                                                 'MaxDayTestSinceFirstPOSITIVETest',
                                                                 'MaxDayTestBEFOREFirstPOSITIVETest',
                                                                 #'DaysToNegStatus',
                                                                 #'DaysToNegStatus_sensitiv',
                                                                 #'DaysToNegStatus_biol',
                                                                 #'DaysToNegStatus_sensitiv_biol',
                                                                 'ResultChangeToNegStatus',
                                                                 'ResultChangeToFinalNegStatus',
                                                                 #'Status',
                                                                 #'Status_Change',
                                                                 'AreaFirstTest',
                                                                 #'PostCodeFirstTest',
                                                                 'FacilityFirstTest',
                                                                 #'Status_Num'
                                           ))


In [ ]:
"Number of patients"
nrow(SwabData_PATIENTS)
length(unique(SwabData$Patient_ID))

*******************

## 2. Positive Patients: SwabData_positive

Also to be used for **categorical heatmaps**.

In [ ]:
#Create a cohort for positive tests
#Also to be used for categorical heatmaps (sensitivity)

#get URs of patients tested positive
URpositive <- unique((subset(SwabData, ((SwabData$Result_1 == "YES") )))$Patient_ID)


#create df of all results of positive patients 
SwabData_positive <- SwabData %>% filter(
  Patient_ID %in% URpositive
  )

#sort the df
SwabData_positive <- SwabData_positive[order(SwabData_positive$Patient_ID,
                                               SwabData_positive$DaySinceFirstPOSITIVETest),]

"Number of positive patients"
length(URpositive)
length(unique(SwabData_positive$Patient_ID))

"Number of tests on positive patients"
nrow(SwabData_positive)

In [ ]:
#days since first positive >= zero 

SwabData_positive_AfterPos <- SwabData_positive[SwabData_positive$DaySinceFirstPOSITIVETest >=0, ]

summary(SwabData_positive_AfterPos$DaySinceFirstPOSITIVETest)

### 2.1 Positive Patients - censored after rule is met: SwabData_positive_censored

Remove all test results for those after the first negative status is achieved

Will be used for the **PRIMARY time to event analysis** and **categorical heatmaps**

In [ ]:
#for categorical heatmaps
#DaysToNegStatus = day of clinical status change and contains the final result .: want this row
SwabData_positive_censored <- SwabData_positive[SwabData_positive$DaySinceFirstPOSITIVETest <= SwabData_positive$DaysToNegStatus, ]



"Number of positive patients"
length(unique(SwabData_positive_censored$Patient_ID))
"Number of tests"
nrow(SwabData_positive_censored)


In [ ]:
#days since first positive >= zero 

SwabData_positive_censored_AfterPos <- SwabData_positive_censored[SwabData_positive_censored$DaySinceFirstPOSITIVETest >=0, ]

summary(SwabData_positive_censored_AfterPos$DaySinceFirstPOSITIVETest)

## 2.2 TTE ANALYSIS Positive patients - censored after rule is met for time to event PRIMARY analysis: SwabData_positive_censored_TTE_Primary

In [ ]:
#only keep the final status
SwabData_positive_censored_TTE_Primary <- SwabData_positive[SwabData_positive$DaysToNegStatus == SwabData_positive$DaySinceFirstPOSITIVETest, c('Patient_ID',
                                                                                                                                                 'Age',
                                                                                                                                                 'Sex',
                                                                                                                                                 #'Date_Collected',
                                                                                                                                                 #'Time_collected',
                                                                                                                                                 #'DateTime_Collected',
                                                                                                                                                 #'Result_1', 
                                                                                                                                                 #'Facility_Description',
                                                                                                                                                 'AgeGroup',
                                                                                                                                                 #'PostCode',
                                                                                                                                                 #'Area',
                                                                                                                                                 #'Total.Population',
                                                                                                                                                 #'DaySinceFirstTest',
                                                                                                                                                 #'DaySinceFirstPOSITIVETest',
                                                                                                                                                 #'NumberOfTests',
                                                                                                                                                 #'NumberOfTestsSinceFirstPOSITIVE',
                                                                                                                                                 #'NumberOfTestsBeforeFirstPOSITIVE',
                                                                                                                                                 #'NumberOfNegTests',
                                                                                                                                                 #'NumberOfPosTests',
                                                                                                                                                 #'MaxDayTestSinceFirstPOSITIVETest',
                                                                                                                                                 #'MaxDayTestBEFOREFirstPOSITIVETest',
                                                                                                                                                 #'DaysToNegStatus',
                                                                                                                                                 #'DaysToNegStatus_sensitiv',
                                                                                                                                                 'DaysToNegStatus_biol',
                                                                                                                                                 #'DaysToNegStatus_sensitiv_biol',
                                                                                                                                                 'ResultChangeToNegStatus',
                                                                                                                                                 #'ResultChangeToFinalNegStatus',
                                                                                                                                                 #'Status',
                                                                                                                                                 #'Status_Change',
                                                                                                                                                 #'AreaFirstTest',
                                                                                                                                                 #'PostCodeFirstTest',
                                                                                                                                                 #'FacilityFirstTest',
                                                                                                                                                 'Status_Num'
                                                                                                                                               
                                                                                                                                               )]

"Number of rows"
nrow(SwabData_positive_censored_TTE_Primary)

"Number of patients"
length(unique(SwabData_positive_censored_TTE_Primary$Patient_ID))

"Summary of number of days to neg status"
summary(SwabData_positive_censored_TTE_Primary$DaysToNegStatus_biol)

"*******************************************************"
"Number of events"
count(SwabData_positive_censored_TTE_Primary$Status_Num)

#rename DaysToNegStatus_sensitiv_biol column to Days for easier analysis
names(SwabData_positive_censored_TTE_Primary)[names(SwabData_positive_censored_TTE_Primary) == "DaysToNegStatus_biol"] <- "DaysToNeg"

head(SwabData_positive_censored_TTE_Primary)

#SwabData_positive_censored_TTE_Primary[SwabData_positive_censored_TTE_Primary$Patient_ID == 60210, ]

#SwabData_positive_3plus_censored_TTE_Primary[SwabData_positive_3plus_censored_TTE_Primary$Patient_ID==746,]

In [ ]:
#save df
write.csv(SwabData_positive_censored_TTE_Primary,'Outputs/TimeToEvent/SwabData_positive_censored_TTE_Primary.csv')

## 2.3 TTE ANALYSIS Positive patients - UNcensored after rule is met for time to event SENSITIVITY analysis: SwabData_positive_UNcensored_TTE_Sensitivity

In [ ]:
#DaysToNegStatus = day of clinical status change and contains the final result .: want this row
SwabData_positive_UNcensored_TTE_Sensitivity <- SwabData_positive[SwabData_positive$DaySinceFirstPOSITIVETest == SwabData_positive$DaysToNegStatus_sensitiv, c('Patient_ID',
                                                                                                                                                                 'Age',
                                                                                                                                                                 'Sex',
                                                                                                                                                                 #'Date_Collected',
                                                                                                                                                                 #'Time_collected',
                                                                                                                                                                 #'DateTime_Collected',
                                                                                                                                                                 #'Result_1', 
                                                                                                                                                                 #'Facility_Description',
                                                                                                                                                                 'AgeGroup',
                                                                                                                                                                 #'PostCode',
                                                                                                                                                                 #'Area',
                                                                                                                                                                 #'Total.Population',
                                                                                                                                                                 #'DaySinceFirstTest',
                                                                                                                                                                 #'DaySinceFirstPOSITIVETest',
                                                                                                                                                                 #'NumberOfTests',
                                                                                                                                                                 #'NumberOfTestsSinceFirstPOSITIVE',
                                                                                                                                                                 #'NumberOfTestsBeforeFirstPOSITIVE',
                                                                                                                                                                 #'NumberOfNegTests',
                                                                                                                                                                 #'NumberOfPosTests',
                                                                                                                                                                 #'MaxDayTestSinceFirstPOSITIVETest',
                                                                                                                                                                 #'MaxDayTestBEFOREFirstPOSITIVETest',
                                                                                                                                                                 #'DaysToNegStatus',
                                                                                                                                                                 #'DaysToNegStatus_sensitiv',
                                                                                                                                                                 #'DaysToNegStatus_biol',
                                                                                                                                                                 'DaysToNegStatus_sensitiv_biol',
                                                                                                                                                                 #'ResultChangeToNegStatus',
                                                                                                                                                                 #ResultChangeToFinalNegStatus',
                                                                                                                                                                 #'Status',
                                                                                                                                                                 #'Status_Change',
                                                                                                                                                                 #AreaFirstTest',
                                                                                                                                                                 #'PostCodeFirstTest',
                                                                                                                                                                 #'FacilityFirstTest',
                                                                                                                                                                 'Status_Num'
                                                                                                                                                                 )]


"Number of patients"
length(unique(SwabData_positive_UNcensored_TTE_Sensitivity$Patient_ID))

"Number of rows"
nrow(SwabData_positive_UNcensored_TTE_Sensitivity)

"Summary of number of days"
summary(SwabData_positive_UNcensored_TTE_Sensitivity$DaysToNegStatus_sensitiv_biol)

"*******************************************************"
"Number of events"
count(SwabData_positive_UNcensored_TTE_Sensitivity$Status_Num)

#rename DaysToNegStatus_sensitiv_biol column to Days for easier analysis
names(SwabData_positive_UNcensored_TTE_Sensitivity)[names(SwabData_positive_UNcensored_TTE_Sensitivity) == "DaysToNegStatus_sensitiv_biol"] <- "DaysToNeg"



head(SwabData_positive_UNcensored_TTE_Sensitivity)

In [ ]:
#save df
write.csv(SwabData_positive_UNcensored_TTE_Sensitivity,'Outputs/TimeToEvent/SwabData_positive_TTE_Sensitivity.csv')

## 2.4 Positive patients patient info: SwabData_positive_PATIENTS

In [ ]:
#creates a df of unique positive patients with age, and sex 

SwabData_positive_PATIENTS <- unique(select(SwabData_positive,'Patient_ID',
                                                                 'Age',
                                                                 'Sex',
                                                                 #'Date_Collected',
                                                                 #'Time_collected',
                                                                 #'DateTime_Collected',
                                                                 #'Result_1', 
                                                                 #'Facility_Description',
                                                                 'AgeGroup',
                                                                 #'PostCode',
                                                                 #'Area',
                                                                 #'Total.Population',
                                                                 #'DaySinceFirstTest',
                                                                 #'DaySinceFirstPOSITIVETest',
                                                                 'NumberOfTests',
                                                                 'NumberOfTestsSinceFirstPOSITIVE',
                                                                 'NumberOfTestsBeforeFirstPOSITIVE',
                                                                 'NumberOfNegTests',
                                                                 'NumberOfPosTests',
                                                                 'MaxDayTestSinceFirstPOSITIVETest',
                                                                 'MaxDayTestBEFOREFirstPOSITIVETest',
                                                                 #'DaysToNegStatus',
                                                                 #'DaysToNegStatus_sensitiv',
                                                                 #'DaysToNegStatus_biol',
                                                                 #'DaysToNegStatus_sensitiv_biol',
                                                                 'ResultChangeToNegStatus',
                                                                 'ResultChangeToFinalNegStatus',
                                                                 #'Status',
                                                                 #'Status_Change',
                                                                 'AreaFirstTest',
                                                                 #'PostCodeFirstTest',
                                                                 'FacilityFirstTest',
                                                                 #'Status_Num'
                                           ))


"Number of positive patients"
nrow(SwabData_positive_PATIENTS)

# 2.5 Positive patients that finished with a Neg status (2 negative tests) Censored at stop rule: SwabData_positive_endNeg_Censored_TTE

In [ ]:
SwabData_positive_endNeg_Censored_TTE <- SwabData_positive_censored_TTE_Primary[SwabData_positive_censored_TTE_Primary$Status_Num == 1, ]

nrow(SwabData_positive_endNeg_Censored_TTE)

# 2.6 Positive patients that finished with a Neg status (2 negative tests) NOT Censored at stop rule: SwabData_positive_endNeg_UNCensored_TTE

In [ ]:
SwabData_positive_endNeg_UNCensored_TTE <- SwabData_positive_UNcensored_TTE_Sensitivity[SwabData_positive_UNcensored_TTE_Sensitivity$Status_Num == 1, ]

nrow(SwabData_positive_endNeg_UNCensored_TTE)

# 2.7 Positive patients that finished with a neg status (2 negative tests OR final neg test but no subsequent tests) Censored at stop rule: SwabData_positive_endNeg_Censored_NEWRULE_TTE

In [ ]:
#get rows that dont have nulls in the DaysToNegStatus_sensitivCensored column
SwabData_positive_endNeg_Censored_NEWRULE <- SwabData_positive[!is.na(SwabData_positive$DaysToNegStatus_sensitivCensored), ]

#DaysToNegStatus = day of clinical status change and contains the final result .: want this row
SwabData_positive_endNeg_Censored_NEWRULE_TTE <- unique(select(SwabData_positive_endNeg_Censored_NEWRULE,'Patient_ID',
                                                                 'Age',
                                                                 'Sex',
                                                                 #'Date_Collected',
                                                                 #'Time_collected',
                                                                 #'DateTime_Collected',
                                                                 #'Result_1', 
                                                                 #'Facility_Description',
                                                                 'AgeGroup',
                                                                 #'PostCode',
                                                                 #'Area',
                                                                 #'Total.Population',
                                                                 #'DaySinceFirstTest',
                                                                 #'DaySinceFirstPOSITIVETest',
                                                                 #'NumberOfTests',
                                                                 #'NumberOfTestsSinceFirstPOSITIVE',
                                                                 #'NumberOfTestsBeforeFirstPOSITIVE',
                                                                 #'NumberOfNegTests',
                                                                 #'NumberOfPosTests',
                                                                 #'MaxDayTestSinceFirstPOSITIVETest',
                                                                 #'MaxDayTestBEFOREFirstPOSITIVETest',
                                                                 #'DaysToNegStatus',
                                                                 #'DaysToNegStatus_sensitiv',
                                                                 #'DaysToNegStatus_biol',
                                                                 #'DaysToNegStatus_sensitiv_biol',
                                                                 #'ResultChangeToNegStatus',
                                                                 #'ResultChangeToFinalNegStatus',
                                                                 #'Status',
                                                                 #'Status_Change',
                                                                 'DaysToNegStatus_sensitivCensored',
                                                                 'AreaFirstTest',
                                                                 #'PostCodeFirstTest',
                                                                 #'FacilityFirstTest',
                                                                 #'Status_Num'
                                           ))



#change status num to 1 as by definition these are people who achieved a neg status
SwabData_positive_endNeg_Censored_NEWRULE_TTE$Status_Num <- 1



"Number of patients"
length(unique(SwabData_positive_endNeg_Censored_NEWRULE_TTE$Patient_ID))

"Number of rows"
nrow(SwabData_positive_endNeg_Censored_NEWRULE_TTE)

"Summary of number of days"
summary(SwabData_positive_endNeg_Censored_NEWRULE_TTE$DaysToNegStatus_sensitivCensored)

"*******************************************************"
"Number of events"
count(SwabData_positive_endNeg_Censored_NEWRULE_TTE$Status_Num)

#rename DaysToNegStatus_sensitiv_biol column to Days for easier analysis
names(SwabData_positive_endNeg_Censored_NEWRULE_TTE)[names(SwabData_positive_endNeg_Censored_NEWRULE_TTE) == "DaysToNegStatus_sensitivCensored"] <- "DaysToNeg"



head(SwabData_positive_endNeg_Censored_NEWRULE_TTE)

# 2.8 Positive patients that finished with a neg status (2 negative tests OR final neg test but no subsequent tests) NOT Censored at stop rule: SwabData_positive_endNeg_UNCensored_TTE_NEWRULE

In [ ]:
#get rows that dont have nulls in the DaysToNegStatus_sensitivCensored column
SwabData_positive_endNeg_UNCensored_NEWRULE <- SwabData_positive[!is.na(SwabData_positive$DaysToNegStatus_sensitivUNCensored), ]

#DaysToNegStatus = day of clinical status change and contains the final result .: want this row
SwabData_positive_endNeg_UNCensored_NEWRULE_TTE <- unique(select(SwabData_positive_endNeg_UNCensored_NEWRULE,'Patient_ID',
                                                                 'Age',
                                                                 'Sex',
                                                                 #'Date_Collected',
                                                                 #'Time_collected',
                                                                 #'DateTime_Collected',
                                                                 #'Result_1', 
                                                                 #'Facility_Description',
                                                                 'AgeGroup',
                                                                 #'PostCode',
                                                                 #'Area',
                                                                 #'Total.Population',
                                                                 #'DaySinceFirstTest',
                                                                 #'DaySinceFirstPOSITIVETest',
                                                                 #'NumberOfTests',
                                                                 #'NumberOfTestsSinceFirstPOSITIVE',
                                                                 #'NumberOfTestsBeforeFirstPOSITIVE',
                                                                 #'NumberOfNegTests',
                                                                 #'NumberOfPosTests',
                                                                 #'MaxDayTestSinceFirstPOSITIVETest',
                                                                 #'MaxDayTestBEFOREFirstPOSITIVETest',
                                                                 #'DaysToNegStatus',
                                                                 #'DaysToNegStatus_sensitiv',
                                                                 #'DaysToNegStatus_biol',
                                                                 #'DaysToNegStatus_sensitiv_biol',
                                                                 #'ResultChangeToNegStatus',
                                                                 #'ResultChangeToFinalNegStatus',
                                                                 #'Status',
                                                                 #'Status_Change',
                                                                 'DaysToNegStatus_sensitivUNCensored',
                                                                 'AreaFirstTest',
                                                                 #'PostCodeFirstTest',
                                                                 #'FacilityFirstTest',
                                                                 #'Status_Num'
                                           ))



#change status num to 1 as by definition these are people who achieved a neg status
SwabData_positive_endNeg_UNCensored_NEWRULE_TTE$Status_Num <- 1



"Number of patients"
length(unique(SwabData_positive_endNeg_UNCensored_NEWRULE_TTE$Patient_ID))

"Number of rows"
nrow(SwabData_positive_endNeg_UNCensored_NEWRULE_TTE)

"Summary of number of days"
summary(SwabData_positive_endNeg_UNCensored_NEWRULE_TTE$DaysToNegStatus_sensitivUNCensored)

"*******************************************************"
"Number of events"
count(SwabData_positive_endNeg_UNCensored_NEWRULE_TTE$Status_Num)

#rename DaysToNegStatus_sensitiv_biol column to Days for easier analysis
names(SwabData_positive_endNeg_UNCensored_NEWRULE_TTE)[names(SwabData_positive_endNeg_UNCensored_NEWRULE_TTE) == "DaysToNegStatus_sensitivUNCensored"] <- "DaysToNeg"



head(SwabData_positive_endNeg_UNCensored_NEWRULE_TTE)

*********************************************

## 3. Negative Patients: SwabData_negative

In [ ]:
#Create a df for negative positive tests


`%notin%` <- Negate(`%in%`)


#create df of all results of positive patients 
SwabData_negative <- SwabData %>% filter (
  Patient_ID %notin% URpositive
  
)

#sort the df
SwabData_negative <- SwabData_negative[order(SwabData_negative$Patient_ID,
                                               SwabData_negative$DaySinceFirstPOSITIVETest),]



"Number of negative patients"
length(unique(SwabData_negative$Patient_ID))


## 3.1 Negative patients patient info: SwabData_positive_PATIENTS

In [ ]:
#creates a df of unique positive patients with age, and sex 

SwabData_negative_PATIENTS <- unique(select(SwabData_negative,
                                            'Patient_ID',
                                                                 'Age',
                                                                 'Sex',
                                                                 #'Date_Collected',
                                                                 #'Time_collected',
                                                                 #'DateTime_Collected',
                                                                 #'Result_1', 
                                                                 #'Facility_Description',
                                                                 'AgeGroup',
                                                                 #'PostCode',
                                                                 #'Area',
                                                                 #'Total.Population',
                                                                 #'DaySinceFirstTest',
                                                                 #'DaySinceFirstPOSITIVETest',
                                                                 'NumberOfTests',
                                                                 'NumberOfTestsSinceFirstPOSITIVE',
                                                                 'NumberOfTestsBeforeFirstPOSITIVE',
                                                                 'NumberOfNegTests',
                                                                 'NumberOfPosTests',
                                                                 'MaxDayTestSinceFirstPOSITIVETest',
                                                                 'MaxDayTestBEFOREFirstPOSITIVETest',
                                                                 #'DaysToNegStatus',
                                                                 #'DaysToNegStatus_sensitiv',
                                                                 #'DaysToNegStatus_biol',
                                                                 #'DaysToNegStatus_sensitiv_biol',
                                                                 'ResultChangeToNegStatus',
                                                                 'ResultChangeToFinalNegStatus',
                                                                 #'Status',
                                                                 #'Status_Change',
                                                                 'AreaFirstTest',
                                                                 #'PostCodeFirstTest',
                                                                 #'FacilityFirstTest',
                                                                 #'Status_Num'
                                           ))


"Number of negative patients"
nrow(SwabData_negative_PATIENTS)

******************************************************

## 4. Positive Patients with $\ge$ 3 Swab Results: SwabData_positive_3plus

This contains all the data

Used to **visualise** result and status for all patients reach the end of their data (rather than at first achievement of a neg status).

In [ ]:
#get df of patients only 0+ days after they swabbed positive

URpositive_3plus <- subset(SwabData_positive, ((SwabData_positive$DaySinceFirstPOSITIVETest >= 0)))

#length(unique(URpositive_3plus$Patient_ID))

#count the number of tests each positive patient hadBananaKaka1987

URpositive_count <- count(URpositive_3plus, 
                          "Patient_ID")

#sort the df by number of tests
URpositive_count <- URpositive_count[rev(order(URpositive_count$freq)),]


#get Patient ID of patients with three or more tests after their first positive
URpositive_3plus <- (subset(URpositive_count, (URpositive_count$freq >=3)))$Patient_ID


#create df of all results of patients positive on day 0 and have had more than 3 tests
SwabData_positive_3plus <- SwabData %>% filter(
  Patient_ID %in% URpositive_3plus
  )


"Number of positive patients with three or more tests"
length(URpositive_3plus)
length(unique(SwabData_positive_3plus$Patient_ID))

"Number of rows"
nrow(SwabData_positive_3plus)

head(SwabData_positive_3plus)

## 4.1 Positive patients with $\geq$ 3 swabs patient info: SwabData_positive_3plus_PATIENTS

In [ ]:
SwabData_positive_3plus_PATIENTS  <- unique(select(SwabData_positive_3plus,
                                                     'Patient_ID',
                                                                 'Age',
                                                                 'Sex',
                                                                 #'Date_Collected',
                                                                 #'Time_collected',
                                                                 #'DateTime_Collected',
                                                                 #'Result_1', 
                                                                 #'Facility_Description',
                                                                 'AgeGroup',
                                                                 #'PostCode',
                                                                 #'Area',
                                                                 #'Total.Population',
                                                                 #'DaySinceFirstTest',
                                                                 #'DaySinceFirstPOSITIVETest',
                                                                 'NumberOfTests',
                                                                 #'NumberOfTestsSinceFirstPOSITIVE',
                                                                 #'NumberOfTestsBeforeFirstPOSITIVE',
                                                                 'NumberOfNegTests',
                                                                 'NumberOfPosTests',
                                                                 'MaxDayTestSinceFirstPOSITIVETest',
                                                                 'MaxDayTestBEFOREFirstPOSITIVETest',
                                                                 #'DaysToNegStatus',
                                                                 #'DaysToNegStatus_sensitiv',
                                                                 #'DaysToNegStatus_biol',
                                                                 #'DaysToNegStatus_sensitiv_biol',
                                                                 'ResultChangeToNegStatus',
                                                                 'ResultChangeToFinalNegStatus',
                                                                 #'Status',
                                                                 #'Status_Change',
                                                                 'AreaFirstTest',
                                                                 #'PostCodeFirstTest',
                                                                 #'FacilityFirstTest',
                                                                 #'Status_Num'
                                           ))


"Number of positive patients with three or more tests"
nrow(SwabData_positive_3plus_PATIENTS)

********************************

## 4.2 Positive Patients with $\geq$ 3 Swab Results - censored after rule is met: SwabData_positive_3plus_censored

Remove all test results for those after the first negative status is achieved

Will be used for the **PRIMARY time to event analysis**

In [ ]:
SwabData_positive_3plus_censored <- SwabData_positive_3plus[SwabData_positive_3plus$DaySinceFirstPOSITIVETest <= SwabData_positive_3plus$DaysToNegStatus, ]


length(unique(SwabData_positive_3plus_censored$Patient_ID))
nrow(SwabData_positive_3plus_censored)

## 5. Patients that changed status from NEG back to POS: SwabData_status_Change

In [ ]:
#get URs of patients tested with a status change
URSwabData_status_Change <- unique((subset(SwabData, ((SwabData$Status_Change == 2) )))$Patient_ID)



#create df of all results of positive patients 
SwabData_status_Change <- SwabData %>% filter(
  Patient_ID %in% URSwabData_status_Change
  )

#sort the df
SwabData_status_Change <- SwabData_status_Change[order(SwabData_status_Change$Patient_ID,
                                               SwabData_status_Change$DaySinceFirstPOSITIVETest),]


"Number of patients with status change POS -- NEG -- POS"
length(URSwabData_status_Change)


length(unique(SwabData_status_Change$Patient_ID))

## 5.1 Positive patients that switched from NEG to POS status patient info: SwabData_status_Change_PATIENTS

In [ ]:
#creates a df of unique positive patients with status change with age, and sex 

SwabData_status_Change_PATIENTS <- unique(select(SwabData_status_Change,
                                         'Patient_ID',
                                                                 'Age',
                                                                 'Sex',
                                                                 #'Date_Collected',
                                                                 #'Time_collected',
                                                                 #'DateTime_Collected',
                                                                 #'Result_1', 
                                                                 #'Facility_Description',
                                                                 'AgeGroup',
                                                                 #'PostCode',
                                                                 #'Area',
                                                                 #'Total.Population',
                                                                 #'DaySinceFirstTest',
                                                                 #'DaySinceFirstPOSITIVETest',
                                                                 'NumberOfTests',
                                                                 'NumberOfTestsSinceFirstPOSITIVE',
                                                                 'NumberOfTestsBeforeFirstPOSITIVE',
                                                                 'NumberOfNegTests',
                                                                 'NumberOfPosTests',
                                                                 'MaxDayTestSinceFirstPOSITIVETest',
                                                                 'MaxDayTestBEFOREFirstPOSITIVETest',
                                                                 #'DaysToNegStatus',
                                                                 #'DaysToNegStatus_sensitiv',
                                                                 #'DaysToNegStatus_biol',
                                                                 #'DaysToNegStatus_sensitiv_biol',
                                                                 'ResultChangeToNegStatus',
                                                                 'ResultChangeToFinalNegStatus',
                                                                 #'Status',
                                                                 #'Status_Change',
                                                                 'AreaFirstTest',
                                                                 #'PostCodeFirstTest',
                                                                 #'FacilityFirstTest',
                                                                 #'Status_Num'
                                           ))

nrow(SwabData_status_Change_PATIENTS)

## 6. Positive patients that had tests prior to testing POS

In [ ]:
#get URs of patients tested with a status change
URSwabData_prior_test <- unique((subset(SwabData_positive, ((SwabData_positive$DaySinceFirstPOSITIVETest <0) )))$Patient_ID)



#create df of all results of positive patients 
SwabData_prior_test <- SwabData %>% filter(
  Patient_ID %in% URSwabData_prior_test
  )

#sort the df
SwabData_prior_test <- SwabData_prior_test[order(SwabData_prior_test$Patient_ID,
                                               SwabData_prior_test$DaySinceFirstPOSITIVETest),]


"Number of patients with tests prior to first positive"
length(URSwabData_prior_test)


length(unique(SwabData_prior_test$Patient_ID))

## 6.1 Positive patients that had tests prior to testing POS patient info: SwabData_prior_test_PATIENTS

In [ ]:
#creates a df of unique positive patients with status change with age, and sex 

SwabData_prior_test_PATIENTS <- unique(select(SwabData_prior_test,
                                         'Patient_ID',
                                                                 'Age',
                                                                 'Sex',
                                                                 #'Date_Collected',
                                                                 #'Time_collected',
                                                                 #'DateTime_Collected',
                                                                 #'Result_1', 
                                                                 #'Facility_Description',
                                                                 'AgeGroup',
                                                                 #'PostCode',
                                                                 #'Area',
                                                                 #'Total.Population',
                                                                 #'DaySinceFirstTest',
                                                                 #'DaySinceFirstPOSITIVETest',
                                                                 'NumberOfTests',
                                                                 'NumberOfTestsSinceFirstPOSITIVE',
                                                                 'NumberOfTestsBeforeFirstPOSITIVE',
                                                                 'NumberOfNegTests',
                                                                 'NumberOfPosTests',
                                                                 'MaxDayTestSinceFirstPOSITIVETest',
                                                                 'MaxDayTestBEFOREFirstPOSITIVETest',
                                                                 #'DaysToNegStatus',
                                                                 #'DaysToNegStatus_sensitiv',
                                                                 #'DaysToNegStatus_biol',
                                                                 #'DaysToNegStatus_sensitiv_biol',
                                                                 'ResultChangeToNegStatus',
                                                                 'ResultChangeToFinalNegStatus',
                                                                 #'Status',
                                                                 #'Status_Change',
                                                                 'AreaFirstTest',
                                                                 #'PostCodeFirstTest',
                                                                 #'FacilityFirstTest',
                                                                 #'Status_Num'
                                           ))

nrow(SwabData_prior_test_PATIENTS)

*****************
# **Descriptive Analysis**

In [ ]:
#NUMBER OF POSITIVE PATIENTS

"Number of patients tested"

a <- nrow(SwabData_PATIENTS)
a

"Number of positive patients"
b <- nrow(SwabData_positive_PATIENTS)
b





"% positive patients"
test<- binom.test(b, 
           a,
           0.5,
           alternative="two.sided",
           conf.level=0.95)


round(test$estimate * 100, 3)

"confidence interval"
round(test$conf.int[1] * 100, 3)
round(test$conf.int[2] * 100, 3)

## Sex

In [ ]:
# https://rcompanion.org/handbook/H_02.html
# https://www.rdocumentation.org/packages/DescTools/versions/0.99.36


#create table of frequencies

AllPatients_Sex <- data.frame(table(SwabData_PATIENTS$Sex))


NegPatients_Sex <- data.frame(table(SwabData_negative_PATIENTS$Sex))


PosPatients_Sex <- data.frame(table(SwabData_positive_PATIENTS$Sex))


PosPatients3Plus_Sex <- data.frame(table(SwabData_positive_3plus_PATIENTS$Sex))



PosPatientsNegFirst <- data.frame(table(SwabData_prior_test_PATIENTS$Sex))

#patients that switched
Results_switch_PATIENTS <- SwabData_positive_3plus_PATIENTS[SwabData_positive_3plus_PATIENTS$ResultChangeToNegStatus > 0, ]
PosPatientsChangeResults <- data.frame(table(Results_switch_PATIENTS$Sex))


PosPatientsEndNeg <- data.frame(table(SwabData_positive_endNeg_Censored_TTE$Sex))


PosPatientsChangeStatus_Sex <- data.frame(table(SwabData_status_Change_PATIENTS$Sex))



sexes <- list(AllPatients_Sex, 
              NegPatients_Sex,
              PosPatients_Sex,
              PosPatients3Plus_Sex,
              PosPatientsNegFirst,
              PosPatientsChangeResults,
              PosPatientsEndNeg,
              PosPatientsChangeStatus_Sex
             )

In [ ]:
AllPatients_Sex

In [ ]:
NegPatients_Sex$Freq[1]

In [ ]:
#create df of sexes

DemographicSexes <- data.frame(matrix(ncol = 9, nrow = 3))

names(DemographicSexes)<- c("Name",
                            "All Individuals n (%) (95% CI)",
                            "Negative individuals (%) (95% CI)",
                            "Positive individuals (%) (95% CI)",
                            "Positive individuals with 2 or more swabs after first positive (%) (95% CI)",
                            "Positive individuals that initially tested negative (%) (95% CI)",
                            "Positive individuals that had a negative test followed by a positive (%) (95% CI)",
                            "Positive individuals who achieved a negative status (2 neg tests) (%) (95% CI)", 
                             "Positive individuals with status change from negative back to positive (%) (95% CI)"
                           )

DemographicSexes$Name <- c("Female",
                           "Male",
                           "Not reported") 

#DemographicSexes

In [ ]:
sexes[1]$Freq

In [ ]:
i = 2

for (sex in sexes) {
    
    observed = sex$Freq
    
    #find pt estimate and 95% confidence interval
    #round to 2 significant figures
    
    data <- data.frame(MultinomCI(observed,
           conf.level=0.95,
           method="sisonglaz"))
    
    DemographicSexes[,i] <- paste(observed,  
                                  " (", 
                                  round(data$est*100,2),
                                  "; ",
                                  round(data$'lwr.ci'*100,2), 
                                  ", ", 
                                  round(data$'upr.ci'*100,2), 
                                  ")" ,
                                  sep = "")
    
    i = i+1
    
}


DemographicSexes

In [ ]:
names(SwabData_PATIENTS)

### Difference in male and female positivity rates - OR

In [ ]:
#REF: https://data.library.virginia.edu/comparing-proportions-with-relative-risk-and-odds-ratios/
#create matrix of male/female pos/neg

FemNeg <- nrow(SwabData_negative_PATIENTS[SwabData_negative_PATIENTS$Sex == "F",])
FemNeg

FemPos <- nrow(SwabData_positive_PATIENTS[SwabData_positive_PATIENTS$Sex == "F",])
FemPos



MaleNeg <- nrow(SwabData_negative_PATIENTS[SwabData_negative_PATIENTS$Sex == "M",])
MaleNeg

MalePos <- nrow(SwabData_positive_PATIENTS[SwabData_positive_PATIENTS$Sex == "M",])
MalePos



SexOR <- matrix(c(FemPos, MalePos, FemNeg, MaleNeg), nrow = 2)
dimnames(SexOR) <- list("Group" = c("Female","Male"), "MI" = c("Pos","Neg"))

SexOR


#calculate OR
OR <- oddsratio(SexOR, rev="b")

round(OR$measure, 3)



In [ ]:
#REF: https://data.library.virginia.edu/comparing-proportions-with-relative-risk-and-odds-ratios/
#create matrix of male/female pos/neg

FemNeg <- nrow(SwabData_positive_endNeg_Censored_TTE[SwabData_positive_endNeg_Censored_TTE$Sex == "F",])
FemNeg

FemPos <- nrow(SwabData_positive_3plus_PATIENTS[SwabData_positive_3plus_PATIENTS$Sex == "F",]) - FemNeg
FemPos



MaleNeg <- nrow(SwabData_positive_endNeg_Censored_TTE[SwabData_positive_endNeg_Censored_TTE$Sex == "M",])
MaleNeg

MalePos <- nrow(SwabData_positive_3plus_PATIENTS[SwabData_positive_3plus_PATIENTS$Sex == "M",]) - MaleNeg
MalePos



SexOR <- matrix(c(FemNeg, MaleNeg, FemPos, MalePos), nrow = 2)



dimnames(SexOR) <- list("Group" = c("Female","Male"), "MI" = c("FinalNeg","FinalPos"))

SexOR


#calculate OR
OR <- oddsratio(SexOR, rev="b")

round(OR$measure, 3)



# AGE

In [ ]:
summary(SwabData[SwabData$Age != -5, ]$Age)

In [ ]:
Age = SwabData[SwabData$Age != -5, ]$Age

Q <- quantile(Age, probs=c(.25, .75), na.rm = FALSE)
Q

iqr <- IQR(Age)

iqr

summary(subset(SwabData[SwabData$Age != -5, ], SwabData[SwabData$Age != -5, ]$Age > (Q[1] - 1.5*iqr) & SwabData[SwabData$Age != -5, ]$Age < (Q[2]+1.5*iqr))$Age)

In [ ]:
#find the oldest person after the 120 - this appears to be a mistake
sort(Age, decreasing = TRUE)[1:10]

In [ ]:
summary(SwabData_PATIENTS[(SwabData_PATIENTS$Age != -5) & (SwabData_PATIENTS$Age != 120), ]$Age)

In [ ]:
"All patients"
AllPatients_Age <- SwabData_PATIENTS[(SwabData_PATIENTS$Age != -5) & (SwabData_PATIENTS$Age != 120), ]$Age

#AllPatients_Age

"Negative Patients"
NegPatients_Age <- SwabData_negative_PATIENTS[(SwabData_negative_PATIENTS$Age != -5) & (SwabData_negative_PATIENTS$Age != 120), ]$Age
#NegPatients_Age

"Positive Patients"
PosPatients_Age <- SwabData_positive_PATIENTS[(SwabData_positive_PATIENTS$Age != -5) & (SwabData_positive_PATIENTS$Age != 120), ]$Age
#PosPatients_Age

"Positive Patients with more than 3 swabs"
PosPatients3Plus_Age <- SwabData_positive_3plus_PATIENTS[(SwabData_positive_3plus_PATIENTS$Age != -5) & (SwabData_positive_3plus_PATIENTS$Age != 120), ]$Age
#PosPatients3Plus_Age

"Positive Patients with that tested negative first"
PosPatientsNegFirst_Age <- SwabData_prior_test_PATIENTS [(SwabData_prior_test_PATIENTS$Age != -5) & (SwabData_prior_test_PATIENTS$Age != 120), ]$Age
#PosPatientsNegFirst_Age


"Positive Patients with result change from neg to pos"
PosPatientsChangeResults_Age <- Results_switch_PATIENTS[(Results_switch_PATIENTS$Age != -5) & (Results_switch_PATIENTS$Age != 120), ]$Age
#PosPatientsChangeResults_Age

"Positive Patients that finished with a neg status (2 neg tests)"
PosPatientsEndNeg_Age <- SwabData_positive_endNeg_Censored_TTE[(SwabData_positive_endNeg_Censored_TTE$Age != -5) & (SwabData_positive_endNeg_Censored_TTE$Age != 120), ]$Age
#PosPatientsEndNeg_Age

"Positive Patients with status change"
PosPatientsChangeStatus_Age <- SwabData_status_Change_PATIENTS[(SwabData_status_Change_PATIENTS$Age != -5) & (SwabData_status_Change_PATIENTS$Age != 120), ]$Age
#PosPatientsChangeStatus_Age




ages <- list(AllPatients_Age, 
              NegPatients_Age,
              PosPatients_Age,
              PosPatients3Plus_Age,
              PosPatientsNegFirst_Age,
              PosPatientsChangeResults_Age,
              PosPatientsEndNeg_Age,
              PosPatientsChangeStatus_Age
             )

In [ ]:
#create df of ages

DemographicAges <- data.frame(matrix(ncol = 9, nrow = 1))

names(DemographicAges)<- c("Name",
                            "All Individuals n (%) (95% CI)",
                            "Negative individuals (%) (95% CI)",
                            "Positive individuals (%) (95% CI)",
                            "Positive individuals with 2 or more swabs after first positive (%) (95% CI)",
                            "Positive individuals that initially tested negative (%) (95% CI)",
                            "Positive individuals that had a negative test followed by a positive (%) (95% CI)",
                            "Positive individuals who achieved a negative status (2 neg tests) (%) (95% CI)", 
                             "Positive individuals with status change from negative back to positive (%) (95% CI)"
                           )

DemographicAges$Name <- c("Median Age (Range)") 

DemographicAges

In [ ]:
i = 2

for (age in ages) {
    
    
    DemographicAges[,i] <- paste(median(age), 
                                  " (", 
                                  min(age), 
                                  ", ", 
                                  max(age), 
                                  ")",
                                  sep = ""
                                 )
    
    i = i+1
    
}


DemographicAges

In [ ]:
DemographicData <- rbind(DemographicSexes, 
             DemographicAges)

DemographicData

## Age groups

In [ ]:
# https://rcompanion.org/handbook/H_02.html
# https://www.rdocumentation.org/packages/DescTools/versions/0.99.36


#create table of frequencies

AllPatients_AgeGroup <- data.frame(table(SwabData_PATIENTS$AgeGroup))


NegPatients_AgeGroup <- data.frame(table(SwabData_negative_PATIENTS$AgeGroup))


PosPatients_AgeGroup <- data.frame(table(SwabData_positive_PATIENTS$AgeGroup))


PosPatients3Plus_AgeGroup <- data.frame(table(SwabData_positive_3plus_PATIENTS$AgeGroup))



PosPatientsNegFirst_AgeGroup <- data.frame(table(SwabData_prior_test_PATIENTS$AgeGroup))

#patients that switched
Results_switch_PATIENTS <- SwabData_positive_3plus_PATIENTS[SwabData_positive_3plus_PATIENTS$ResultChangeToNegStatus > 0, ]
PosPatientsChangeResults_AgeGroup <- data.frame(table(Results_switch_PATIENTS$AgeGroup))


PosPatientsEndNeg_AgeGroup <- data.frame(table(SwabData_positive_endNeg_Censored_TTE$AgeGroup))


PosPatientsChangeStatus_AgeGroup <- data.frame(table(SwabData_status_Change_PATIENTS$AgeGroup))



agegroups <- list(AllPatients_AgeGroup, 
              NegPatients_AgeGroup,
              PosPatients_AgeGroup,
              PosPatients3Plus_AgeGroup,
              PosPatientsNegFirst_AgeGroup,
              PosPatientsChangeResults_AgeGroup,
              PosPatientsEndNeg_AgeGroup,
              PosPatientsChangeStatus_AgeGroup
             )

In [ ]:
AllPatients_AgeGroup$Var1

In [ ]:
#create df of age groups

DemographicAgeGroups <- data.frame(matrix(ncol = 9, nrow = 4))

names(DemographicAgeGroups)<- c("Name",
                            "All Individuals n (%) (95% CI)",
                            "Negative individuals (%) (95% CI)",
                            "Positive individuals (%) (95% CI)",
                            "Positive individuals with 2 or more swabs after first positive (%) (95% CI)",
                            "Positive individuals that initially tested negative (%) (95% CI)",
                            "Positive individuals that had a negative test followed by a positive (%) (95% CI)",
                            "Positive individuals who achieved a negative status (2 neg tests) (%) (95% CI)", 
                             "Positive individuals with status change from negative back to positive (%) (95% CI)"
                           )

DemographicAgeGroups$Name <- c("16 and under",
                               "17 to 64",
                               "65 and over",
                               "Not reported"
                              ) 

#DemographicAgeGroups

In [ ]:
i = 2

for (agegroup in agegroups) {
    
    observed = agegroup$Freq
    
    #find pt estimate and 95% confidence interval
    #round to 2 significant figures
    
    data <- data.frame(MultinomCI(observed,
           conf.level=0.95,
           method="sisonglaz"))
    
    DemographicAgeGroups[,i] <- paste(observed,  
                                  " (", 
                                  round(data$est*100,2),
                                  "; ",
                                  round(data$'lwr.ci'*100,2), 
                                  ", ", 
                                  round(data$'upr.ci'*100,2), 
                                  ")" ,
                                  sep = "")
    
    i = i+1
    
}


DemographicAgeGroups

In [ ]:
DemographicData <- rbind(DemographicData, 
             DemographicAgeGroups)

DemographicData

In [ ]:
colnames(DemographicData)

In [ ]:
MainDemographicData <- DemographicData[, c("Name",
                            "All Individuals n (%) (95% CI)",
                            "Negative individuals (%) (95% CI)",
                            "Positive individuals (%) (95% CI)"
                            #"Positive individuals with 2 or more swabs after first positive (%) (95% CI)",
                            #"Positive individuals that initially tested negative (%) (95% CI)",
                            #"Positive individuals that had a negative test followed by a positive (%) (95% CI)",
                            #"Positive individuals who achieved a negative status (2 neg tests) (%) (95% CI)", 
                            # "Positive individuals with status change from negative back to positive (%) (95% CI)"
                           ) ]

MainDemographicData

In [ ]:
SuppDemographicData <- DemographicData[, c("Name",
                            #"All Individuals n (%) (95% CI)",
                            #"Negative individuals (%) (95% CI)",
                            #"Positive individuals (%) (95% CI)"
                            "Positive individuals with 2 or more swabs after first positive (%) (95% CI)",
                            "Positive individuals that initially tested negative (%) (95% CI)",
                            #"Positive individuals that had a negative test followed by a positive (%) (95% CI)",
                            "Positive individuals who achieved a negative status (2 neg tests) (%) (95% CI)", 
                             "Positive individuals with status change from negative back to positive (%) (95% CI)"
                           ) ]

SuppDemographicData

### Difference in AgeGroup positivity rates - OR

In [ ]:
#REF: https://data.library.virginia.edu/comparing-proportions-with-relative-risk-and-odds-ratios/
#create matrix of age groups

USixteenNeg <- nrow(SwabData_negative_PATIENTS[SwabData_negative_PATIENTS$AgeGroup == "16 and under",])
USixteenNeg

USixteenPos <- nrow(SwabData_positive_PATIENTS[SwabData_positive_PATIENTS$AgeGroup == "16 and under",])
USixteenPos



SevenTeenSixyFourNeg <- nrow(SwabData_negative_PATIENTS[SwabData_negative_PATIENTS$AgeGroup == "17 to 64",])
SevenTeenSixyFourNeg

SevenTeenSixyFourPos <- nrow(SwabData_positive_PATIENTS[SwabData_positive_PATIENTS$AgeGroup == "17 to 64",])
SevenTeenSixyFourPos



AgeGrpOR <- matrix(c(SevenTeenSixyFourPos,USixteenPos,  SevenTeenSixyFourNeg, USixteenNeg), nrow = 2)
dimnames(AgeGrpOR) <- list("Group" = c("17 to 64","16 and under"), "MI" = c("Pos","Neg"))

AgeGrpOR


#calculate OR
OR <- oddsratio(AgeGrpOR, rev="b")

round(OR$measure, 3)



In [ ]:
#REF: https://data.library.virginia.edu/comparing-proportions-with-relative-risk-and-odds-ratios/
#create matrix of age groups

USixteenNeg <- nrow(SwabData_negative_PATIENTS[SwabData_negative_PATIENTS$AgeGroup == "16 and under",])
USixteenNeg

USixteenPos <- nrow(SwabData_positive_PATIENTS[SwabData_positive_PATIENTS$AgeGroup == "16 and under",])
USixteenPos



OSixtFiveNeg <- nrow(SwabData_negative_PATIENTS[SwabData_negative_PATIENTS$AgeGroup == "65 and over",])
OSixtFiveNeg

OSixtFivePos <- nrow(SwabData_positive_PATIENTS[SwabData_positive_PATIENTS$AgeGroup == "65 and over",])
OSixtFivePos



AgeGrpOR <- matrix(c(OSixtFivePos,USixteenPos,  OSixtFiveNeg, USixteenNeg), nrow = 2)
dimnames(AgeGrpOR) <- list("Group" = c("65 and over","16 and under"), "MI" = c("Pos","Neg"))

AgeGrpOR


#calculate OR
OR <- oddsratio(AgeGrpOR, rev="b")

round(OR$measure, 3)

## TREND in Positivity ~ AgeGroup

In [ ]:
# https://rcompanion.org/handbook/H_02.html
# https://www.rdocumentation.org/packages/DescTools/versions/0.99.36


#create table of frequencies



PosPatients_AgeGroup <- data.frame(table(SwabData_positive_PATIENTS$AgeGroup))

NegPatients_AgeGroup <- data.frame(table(SwabData_negative_PATIENTS$AgeGroup))


agegroups <- list(PosPatients_AgeGroup, 
                  NegPatients_AgeGroup
             )

In [ ]:
#create df of age groups

AgeGroups <- data.frame(matrix(ncol = 3, nrow = 4))

names(AgeGroups)<- c("Name",
                     "Pos",
                     "Neg"
                           )

AgeGroups$Name <- c("16 and under",
                               "17 to 64",
                               "65 and over",
                               "Not reported"
                              ) 



In [ ]:
i = 2

for (agegroup in agegroups) {
    
    observed = agegroup$Freq
    
    
    AgeGroups[,i] <- observed
    
    i = i+1
    
}


AgeGroups

In [ ]:
AgeGroups<- AgeGroups[1:3, ]

AgeGroups

In [ ]:
AgeGroups$Total= AgeGroups$Pos + AgeGroups$Neg
AgeGroups

In [ ]:
chisq.test(AgeGroups[, 2:3])

In [ ]:
#https://stat.ethz.ch/R-manual/R-devel/library/stats/html/prop.trend.test.html
prop.trend.test(AgeGroups$Pos, 
               AgeGroups$Total, 
                score = seq_along(AgeGroups$Pos))

## Area

In [ ]:
#Number of positive cases grouped by area 
Area_Positive = count(SwabData_positive_PATIENTS, "AreaFirstTest")

names(Area_Positive)[1] <- "Area"

names(Area_Positive)[2] <- "Positive Patients"

Area_Positive

sum(Area_Positive$"Positive Patients")

In [ ]:
#Number of screening tests (first test) grouped by area 
Area_All <- count(SwabData_PATIENTS, "AreaFirstTest")

names(Area_All)[1] <- "Area"

names(Area_All)[2] <- "Patients tested"

Area_All

sum(Area_All$"Patients tested")

In [ ]:
#join tables
Area_Test = merge(x = Area_Positive, 
                  y = Area_All, 
                  by = "Area", 
                  all = TRUE)

Area_Test[is.na(Area_Test)] <- 0

#add Very remote and remote autralia
Area_Test[7,  ] <- Area_Test[5, ] + Area_Test[6, ]

#changes names from factors to character
Area_Test$Area <- as.character(Area_Test$Area)

#rename row
Area_Test[7,"Area"] <- "Remote and Very Remote Australia"

Area_Test$Area <- as.factor(Area_Test$Area)

Area_Test <- Area_Test[Area_Test$Area != c("Remote Australia",
                                          "Very Remote Australia"), ]



Area_Test

In [ ]:
i = 1

for (i in 1:nrow(Area_Test)){
    
    k <- Area_Test$'Positive Patients'[i]

    n <- Area_Test$'Patients tested'[i]


    propTest = prop.test(k, n)


    propTest$estimate
    propTest$conf.int[1]
    propTest$conf.int[2]
    
    Area_Test[i, "Percentage positive tests (95% CI)"] <- paste (round(as.numeric(propTest$estimate)*100, 3),
                                                        " (",
                                                        round(propTest$conf.int[1]*100, 3),
                                                        ", ",
                                                        round(propTest$conf.int[2]*100, 3),
                                                        ")",
                                                        sep = ""
                                                       )
    
    i = i+1
    
    
}



Area_Test


In [ ]:
unique(Area_Test$Area)

In [ ]:
x <- c("", 
       "Major Cities of Australia", 
       "Inner Regional Australia",
      "Outer Regional Australia",
      "Remote and Very Remote Australia")

Area_Test %>%
  slice(match(x, Area))

### Facilitites with missing area

In [ ]:
#facilities without area classification
facilities <- unique(SwabData_PATIENTS[SwabData_PATIENTS$AreaFirstTest == unique(SwabData_PATIENTS$AreaFirstTest)[5],  "FacilityFirstTest"])

#create df of all results of positive patients 
data_pos <- SwabData_positive_PATIENTS %>% filter(
  FacilityFirstTest %in% facilities
  )

#unique(data$FacilityFirstTest)


#create df of all results of positive patients 
data <- SwabData_PATIENTS %>% filter(
  FacilityFirstTest %in% facilities
  )

In [ ]:
#Number of positive cases grouped by area 
Area_Positive = count(data_pos, "FacilityFirstTest")

names(Area_Positive)[1] <- "Facility"

names(Area_Positive)[2] <- "Positive Patients"

Area_Positive

sum(Area_Positive$"Positive Patients")

In [ ]:
#Number of screening tests (first test) grouped by area 
Area_All <- count(data, "FacilityFirstTest")

names(Area_All)[1] <- "Facility"

names(Area_All)[2] <- "Patients tested"

Area_All

sum(Area_All$"Patients tested")

In [ ]:
#join tables
Area_Test = merge(x = Area_Positive, 
                  y = Area_All, 
                  by = "Facility", 
                  all = TRUE)

Area_Test[is.na(Area_Test)] <- 0


Area_Test

In [ ]:
i = 1

for (i in 1:nrow(Area_Test)){
    
    k <- Area_Test$'Positive Patients'[i]

    n <- Area_Test$'Patients tested'[i]


    propTest = prop.test(k, n)


    propTest$estimate
    propTest$conf.int[1]
    propTest$conf.int[2]
    
    Area_Test[i, "Percentage positive tests"] <- paste (round(as.numeric(propTest$estimate)*100, 2),
                                                        " (",
                                                        round(propTest$conf.int[1]*100, 2),
                                                        ", ",
                                                        round(propTest$conf.int[2]*100, 2),
                                                        ")",
                                                        sep = ""
                                                       )
    
    i = i+1
    
    
}



Area_Test


## Number of tests

In [ ]:
#Create a df for patients who never became positive


`%notin%` <- Negate(`%in%`)


#create df of all results of positive patients 
NegTests <- all_data_preCalcs %>% filter (
  Patient_ID %notin% URpositive
  
)

"Number of negative patients"
length(unique(NegTests$Patient_ID))


In [ ]:
#NEGATIVE PATIENTS

dataFreq_counts <- count(count(NegTests$Patient_ID)$freq)

#dataFreq_counts

dataFreq <- count(NegTests$Patient_ID)$freq

#hist(count(SwabData_positive_afterPos$Patient_ID)$freq,
 #   main = "Tests (on seperate days) per patient",
  #  xlab = "Number of tests",
   # ylab = "Number of patients")



#dataFreq
br = c(0 , 1, 2, 100)

ranges = c(1, 
           2,
           "3 or more")


freqNeg <- hist(dataFreq, 
              breaks=br, 
              include.lowest=TRUE, 
              plot=FALSE)

freqNeg<- data.frame(range = ranges, frequency = freqNeg$counts)

names(freqNeg) <- c("Number of tests",
                            "Negative patients"
                           )

"Number of patients"
sum(freqNeg$'Negative patients')

maxTest <- data.frame("Max tests",
                      max(dataFreq_counts$x))

names(maxTest) <- c("Number of tests",
                            "Negative patients"
                           )



freqNeg <- rbind(freqNeg, maxTest)




freqNeg


In [ ]:
#POSITIVE PATIENTS

SwabData_positive_afterPos <- SwabData_positive[SwabData_positive$DaySinceFirstPOSITIVETest > 0, ]


dataFreq_counts <- count(count(SwabData_positive_afterPos$Patient_ID)$freq)


dataFreq <- count(SwabData_positive_afterPos$Patient_ID)$freq

#hist(count(SwabData_positive_afterPos$Patient_ID)$freq,
 #   main = "Tests (on seperate days) per patient",
  #  xlab = "Number of tests",
   # ylab = "Number of patients")



br = c(0 , 1, 2, 100)

ranges = c(1, 
           2,
           "3 or more")


freqPos <- hist(dataFreq, 
              breaks=br, 
              include.lowest=TRUE, 
              plot=FALSE)

freqPos<- data.frame( range = ranges, 
                     frequency = freqPos$counts)

names(freqPos) <- c("Number of tests",
                            "Positive patients: After testing positive"
                           )


"Number of patients with repeat tests"
a <- sum(freqPos$"Positive patients: After testing positive")
a

b <- nrow(SwabData_positive_PATIENTS)


"% patients with repeat tests"
test<- binom.test(a, 
           b,
           0.5,
           alternative="two.sided",
           conf.level=0.95)


round(test$estimate * 100, 3)

"confidence interval"
round(test$conf.int[1] * 100, 3)
round(test$conf.int[2] * 100, 3)


"*****************"

"number with 3 or more tests"
a <- nrow(SwabData_positive_3plus_PATIENTS)
a

"% patients with 2 or more tests"
test<- binom.test(a, 
           b,
           0.5,
           alternative="two.sided",
           conf.level=0.95)


round(test$estimate * 100, 3)

"confidence interval"
round(test$conf.int[1] * 100, 3)
round(test$conf.int[2] * 100, 3)


"*****************"



maxTest <- data.frame("Max tests",
                      max(dataFreq_counts$x))

names(maxTest) <- c("Number of tests",
                            "Positive patients: After testing positive"
                           )



freqPos <- rbind(freqPos, maxTest)




freqPos




In [ ]:
#PRIOR TO POS PATIENTS

SwabData_positive_afterPos <- SwabData_positive[SwabData_positive$DaySinceFirstPOSITIVETest < 0, ]


dataFreq_counts <- count(count(SwabData_positive_afterPos$Patient_ID)$freq)



dataFreq <- count(SwabData_positive_afterPos$Patient_ID)$freq

#hist(count(SwabData_positive_afterPos$Patient_ID)$freq,
 #   main = "Tests (on seperate days) per patient",
  #  xlab = "Number of tests",
   # ylab = "Number of patients")



br = c(0 , 1, 2, 100)

ranges = c(1, 
           2,
           "3 or more")


freqPriorPos <- hist(dataFreq, 
              breaks=br, 
              include.lowest=TRUE, 
              plot=FALSE)

freqPriorPos<- data.frame(range = ranges, 
                     frequency = freqPriorPos$counts)

names(freqPriorPos) <- c("Number of tests",
                            "Positive patients: Prior to testing positive"
                           )




maxTest <- data.frame("Max tests",
                      max(dataFreq_counts$x))

names(maxTest) <- c("Number of tests",
                            "Positive patients: Prior to testing positive"
                           )



freqPriorPos <- rbind(freqPriorPos, maxTest)






freqPriorPos




In [ ]:
# JOIN TABLES

TestingNumber <-  merge(x = freqNeg, 
                  y = freqPos, 
                  by = "Number of tests", 
                  all = TRUE)



TestingNumber <- merge(x = TestingNumber, 
                  y = freqPriorPos, 
                  by = "Number of tests",  
                  all = TRUE)

TestingNumber

In [ ]:
#NUMBER OF POSITIVE PATIENTS with >= 2 tests

"Number of positive patients"


a <- nrow(SwabData_positive_PATIENTS)
a

"Number of positive patients with repeat tests"
b <- sum(TestingNumber$"Positive patients: After testing positive"[1:3])
b





"% positive patients"
test<- binom.test(b, 
           a,
           0.5,
           alternative="two.sided",
           conf.level=0.95)


round(test$estimate * 100, 3)

"confidence interval"
round(test$conf.int[1] * 100, 3)
round(test$conf.int[2] * 100, 3)


In [ ]:
#NUMBER OF POSITIVE PATIENTS with >=3 tests

"Number of positive patients"


a <- nrow(SwabData_positive_PATIENTS)
a

"Number of positive patients with 2 or more repeat tests"
b <- nrow(SwabData_positive_3plus_PATIENTS)
b





"% positive patients"
test<- binom.test(b, 
           a,
           0.5,
           alternative="two.sided",
           conf.level=0.95)


round(test$estimate * 100, 3)

"confidence interval"
round(test$conf.int[1] * 100, 3)
round(test$conf.int[2] * 100, 3)

In [ ]:
#NUMBER OF POSITIVE PATIENTS with >=3 tests and NEG STATUS

"Number of positive patients"


a <- nrow(SwabData_positive_3plus_PATIENTS)
a

"Number of positive patients with 3 or more tests"
b <- nrow(SwabData_positive_endNeg_Censored_TTE)
b





"% positive patients"
test<- binom.test(b, 
           a,
           0.5,
           alternative="two.sided",
           conf.level=0.95)


round(test$estimate * 100, 3)

"confidence interval"
round(test$conf.int[1] * 100, 3)
round(test$conf.int[2] * 100, 3)

In [ ]:
nrow(SwabData_positive_endNeg_Censored_TTE)

### Tests in positive patients

In [ ]:
SwabData_positive_afterPos <- SwabData_positive[SwabData_positive$DaySinceFirstPOSITIVETest >= 0, ]

length(unique(SwabData_positive$Patient_ID))
length(unique(SwabData_positive_afterPos$Patient_ID))


max(SwabData_positive_afterPos$DaySinceFirstPOSITIVETest)

In [ ]:
#count(count(SwabData_positive_afterPos$Patient_ID)$freq)

dataFreq <- count(SwabData_positive_afterPos$Patient_ID)

#hist(count(SwabData_positive_afterPos$Patient_ID)$freq,
 #   main = "Tests (on seperate days) per patient",
  #  xlab = "Number of tests",
   # ylab = "Number of patients")


#dataFreq

histFreqTest <- ggplot(dataFreq, 
                       aes(x = freq)) + 

geom_histogram(color="black", fill="white") +

#change titles
labs(title = "Number of seperate days of tests per patient\non or after testing positive for COVID19", #title
     subtitle = "", 
     caption = "", 
     
     x = "Number of tests",  #x axis
     y = "Number of patients") + #y axis 


theme(
    plot.title = element_text(color = "black", 
                              size = 12,
                              hjust = 0.5)
    )

#histFreqTest

ggsave(histFreqTest,
       file="Outputs/Visuals/PositivePatient_NumberTests.png")

### Tests prior to first positive

In [ ]:
summary(SwabData_prior_test$MaxDayTestBEFOREFirstPOSITIVETest)

"Number of patients with test prior"
a <- length(unique(SwabData_prior_test$Patient_ID))
a


"Number of positive patients"
b <- length(unique(SwabData_positive_PATIENTS$Patient_ID))
b

"% patients that tested neg and then pos"
test<- binom.test(a, 
           b,
           0.5,
           alternative="two.sided",
           conf.level=0.95)


round(test$estimate * 100, 3)

"confidence interval"
round(test$conf.int[1] * 100, 3)
round(test$conf.int[2] * 100, 3)





"Number of patients with first test outside the 14 day window"
b <- length(unique(SwabData_prior_test[SwabData_prior_test$MaxDayTestBEFOREFirstPOSITIVETest < -14, ]$Patient_ID))
b

"% patient outside the 14 window"
test<- binom.test(b, 
           a,
           0.5,
           alternative="two.sided",
           conf.level=0.95)


round(test$estimate * 100, 3)

"confidence interval"
round(test$conf.int[1] * 100, 3)
round(test$conf.int[2] * 100, 3)

#count(SwabData_prior_test$DaySinceFirstPOSITIVETest)

"Number tests prior to first positive"
a <- nrow(SwabData_prior_test)
a

"Number tests 14 days prior to first positive"
b <- nrow(SwabData_prior_test[SwabData_prior_test$DaySinceFirstPOSITIVETest < -14, ])
b





In [ ]:
SwabData_positive_beforePos <- SwabData[SwabData$DaySinceFirstPOSITIVETest < 0, ]

length(unique(SwabData_positive_beforePos$Patient_ID))
length(unique(SwabData_positive_beforePos$Patient_ID))


count(count(SwabData_positive_beforePos$Patient_ID)$freq)

dataFreq <- count(SwabData_positive_beforePos$Patient_ID)

#hist(count(SwabData_positive_afterPos$Patient_ID)$freq,
 #   main = "Tests (on seperate days) per patient",
  #  xlab = "Number of tests",
   # ylab = "Number of patients")


#dataFreq

histFreqTest <- ggplot(dataFreq, 
                       aes(x = freq)) + 

geom_histogram(color="black", fill="white") +

#change titles
labs(title = "Number of seperate days of tests per patient\nbefore testing positive for COVID19", #title
     subtitle = "", 
     caption = "", 
     
     x = "Number of tests",  #x axis
     y = "Number of patients") + #y axis 


theme(
    plot.title = element_text(color = "black", 
                              size = 12,
                              hjust = 0.5)
    )

#histFreqTest

ggsave(histFreqTest,
       file="Outputs/Visuals/NegPriorPatient_NumberTests.png")

## TEST RESULTS: Switch from neg to positive test result

In [ ]:
#number of switches to census neg
Switches <- count(SwabData_positive_3plus_PATIENTS$ResultChangeToNegStatus)


names(Switches)[2] <- "Censored"

summary(SwabData_positive_3plus_PATIENTS$ResultChangeToNegStatus)

In [ ]:
#number of switches to final neg

Switches = merge(x = Switches, 
                  y = count(SwabData_positive_3plus_PATIENTS$ResultChangeToFinalNegStatus), 
                  by = "x", 
                  all = TRUE)

summary(SwabData_positive_3plus_PATIENTS$ResultChangeToFinalNegStatus)


In [ ]:
Switches = merge(x = Switches, 
                  y = count(SwabData_status_Change_PATIENTS$ResultChangeToFinalNegStatus), 
                  by = "x", 
                  all = TRUE)

In [ ]:
names(Switches)[1] <- "Number of switches"
names(Switches)[3] <- "Uncensored"
names(Switches)[4] <- "Patients that changed from negative back to postive status"

Switches[is.na(Switches)] <- 0

Switches

# Acheving Neg Status

In [ ]:
"Number that achieved a neg status"
a <- length(unique(SwabData_positive_endNeg_Censored_TTE$Patient_ID))
a

"Number of positive patients with 2 or more repeat tests"
b <- length(unique(SwabData_positive_3plus_PATIENTS$Patient_ID))
b


"% patients that achieved a neg status"
test<- binom.test(a, 
           b,
           0.5,
           alternative="two.sided",
           conf.level=0.95)


round(test$estimate * 100, 3)

"confidence interval"
round(test$conf.int[1] * 100, 3)
round(test$conf.int[2] * 100, 3)

# Patients with Pos status after day 10

In [ ]:
"Number of patients with pos test/status after day 10"

a <- length(SwabData_positive_censored_TTE_Primary[SwabData_positive_censored_TTE_Primary$DaysToNeg > 10,]$Patient_ID)
a

"Number of positive patients with 2 or more repeat tests"
b <- length(unique(SwabData_positive_3plus_PATIENTS$Patient_ID))
b


"% patients that achieved a neg status"
test<- binom.test(a, 
           b,
           0.5,
           alternative="two.sided",
           conf.level=0.95)


round(test$estimate * 100, 3)

"confidence interval"
round(test$conf.int[1] * 100, 3)
round(test$conf.int[2] * 100, 3)

max(SwabData_positive_3plus$DaySinceFirstPOSITIVETest)

# Patients who switched from Neg status back to Pos

In [ ]:
"Number of patients with neg back to pos status"

a <- nrow(SwabData_status_Change_PATIENTS)
a

"Number of positive patients with 2 or more repeat tests"
b <- length(unique(SwabData_positive_endNeg_Censored_TTE$Patient_ID))
b


"% patients that achieved a neg status"
test<- binom.test(a, 
           b,
           0.5,
           alternative="two.sided",
           conf.level=0.95)


round(test$estimate * 100, 2)

"confidence interval"
round(test$conf.int[1] * 100, 2)
round(test$conf.int[2] * 100, 2)

**********************

# Testing Time Series

In [ ]:
#create df of first tests
df <- SwabData[ SwabData$DaySinceFirstPOSITIVETest <= 0, c(#'Patient_ID',
                                                                 #'Age',
                                                                 #'Sex',
                                                                 'Date_Collected',
                                                                 #'Time_collected',
                                                                 #'DateTime_Collected',
                                                                 'Result_1' 
                                                                 #'Facility_Description',
                                                                 #'AgeGroup',
                                                                 #'PostCode',
                                                                 #'Area',
                                                                 #'Total.Population',
                                                                 #'DaySinceFirstTest',
                                                                 #'DaySinceFirstPOSITIVETest',
                                                                 #'NumberOfTests',
                                                                 #'NumberOfTestsSinceFirstPOSITIVE',
                                                                 #'NumberOfTestsBeforeFirstPOSITIVE',
                                                                 #'NumberOfNegTests',
                                                                 #'NumberOfPosTests',
                                                                 #'MaxDayTestSinceFirstPOSITIVETest',
                                                                 #'MaxDayTestBEFOREFirstPOSITIVETest',
                                                                 #'DaysToNegStatus',
                                                                 #'DaysToNegStatus_sensitiv',
                                                                 #'DaysToNegStatus_biol',
                                                                 #'DaysToNegStatus_sensitiv_biol',
                                                                 #'ResultChangeToNegStatus',
                                                                 #'ResultChangeToFinalNegStatus',
                                                                 #'Status',
                                                                 #'Status_Change',
                                                                 #'AreaFirstTest',
                                                                 #'PostCodeFirstTest',
                                                                 #'FacilityFirstTest',
                                                                 #'Status_Num'
                                                          )]

In [ ]:
#number of positive tests on each day
positiveTimeSeries <- count(df[df$Result_1 == "YES", ]$Date_Collected)

names(positiveTimeSeries) <- c("Date",
                            "NumberPositive"
                           )


#total tests on each day
testTimeSeries <- count(df$Date_Collected)

names(testTimeSeries) <- c("Date",
                            "NumberScreened"
                           )



#merge
TestingTimeSeries <- merge(x = testTimeSeries, 
                  y = positiveTimeSeries, 
                  by = "Date",  
                  all = TRUE)



TestingTimeSeries$PercentPost <- (TestingTimeSeries$NumberPositive/TestingTimeSeries$NumberScreened)*100





#convert date column to date format
TestingTimeSeries$Date <- as.Date(TestingTimeSeries$Date, format = "%Y-%m-%d")

#repalce na with 0
TestingTimeSeries[is.na(TestingTimeSeries)] <- 0

is.Date(TestingTimeSeries$Date)


#save df
write.csv(TestingTimeSeries,'Outputs/TimeToEvent/TestingTimeSeries.csv')

In [ ]:
max(TestingTimeSeries$NumberPositive)

max(TestingTimeSeries$NumberScreened)



In [ ]:
wrapper <- function(x, ...) 
{
  paste(strwrap(x, ...), collapse = "\n")
}

In [ ]:
#important dates

labels <-  data.frame(date = c("2020-01-10", #initial testing requirements
                               "2020-03-20", #International border closure
                             #  "2020-03-21", #Change in release criteria
                               "2020-03-25", #State border closure
                               "2020-04-01", #expansion of texting - certain areas
                               "2020-04-30", #expansion of texting - Qld wide
                               "2020-06-04" #no longer require swab to leave iso
                              ),
                      
                     
                      y = c(500,
                            2200,
                         #   2450,
                            2200,
                            2200,
                            500,
                            500
                           )
                      
                      
                      
                     
                     
                     
                     )





labels$date <- as.Date(labels$date)

labels$effect <- labels$date + 14

labels$id <- seq.int(nrow(labels)) 

labels



                      


In [ ]:
#REF: https://stackoverflow.com/questions/51132115/plot-with-ggplot-a-graph-with-two-y-scales
#smoothing: https://ggplot2.tidyverse.org/reference/geom_smooth.html

p <- ggplot(TestingTimeSeries) +


#number screened
geom_line(aes(x=Date, 
                y=NumberScreened,
               group = 1,
              colour = "grey9")
         ) + 


#SMOOTHING
stat_smooth(aes(x=Date, 
                y=NumberScreened,
              colour = "blue"),
            se = FALSE,
            na.rm = FALSE,
            method = "gam",
            formula = y ~ s(x, bs = "cs"),
            #span = 0.3,
           ) +


#y axis
scale_y_continuous(limits=c(0, 2500), 
                           sec.axis = sec_axis(~ . *200/2400, name = "Number of new cases"))+


#labels
labs(
    #title = "Tested* and positive individuals over time",
    x="Date", 
     y="Number individuals tested") +

theme(plot.title = element_text(hjust = 0.5),
     plot.caption = element_text(size = 7,
                                hjust = 0)) +


#number that test positive
geom_line(aes(x=Date, 
                y=NumberPositive*2400/200,
               group = 1,
              colour = "grey9")
         ) + 

#SMOOTHING
stat_smooth(aes(x=Date, 
                y=NumberPositive*2400/200,
                colour = "red"), 
            se = FALSE,
           
           na.rm = FALSE,
           method = "loess", #"gam",
           # formula = y ~ s(x, bs = "cs"),
            span = 0.1,
            show.legend = TRUE
           ) +


#adjust date format
scale_x_date(date_labels = "%d %b %Y") +


#add legend
scale_colour_manual(name = "" ,
                    breaks = c("blue",
                             "red"),
                    labels = c("Number individuals tested (trend line)",
                               "Number of new cases (trend line)"),
                    limits=c("blue",
                             "red",
                             "grey9"),
                    values = c("blue",
                               "red",
                             "grey9")) +

#scale_linetype_manual(values = c("X1977" = 1, "X1978" = 1, "X1979" = 2)) +

#scale_shape_manual(values = c("X1977" = 16, "X1978" = 17, "X1979" = 18))


#legend
theme(legend.position = c(0.2,0.7),
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 


#remove background colour and grid
theme(panel.border = element_blank(), 
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank(), 
      panel.background = element_blank(),
     #line = element_blank()
       axis.line = element_line(colour = "black")
      
     ) +




#CHANGES IN TESTING ELIGIBILIGY
#REF http://www.sthda.com/english/wiki/ggplot2-add-straight-lines-to-a-plot-horizontal-vertical-and-regression-lines

geom_text(labels,
          mapping = aes(x = date,
              y = y,
              label = id)) +



geom_segment(labels,
             mapping = aes(x = date,
                           y = y-50, 
                           xend = date, 
                           yend = 0),
             linetype = "dotdash",
             color = "grey4",
            arrow = arrow(length = unit(0.3, "cm"))) 



ggsave("Outputs/Visuals/TestingTimeSeries_NumScreenPos.png", 
       height = 7 , 
       width = 7 )



p

******************************************************
# Time to Event Analysis

https://rviews.rstudio.com/2017/09/25/survival-analysis-with-r/

[ggsurvplot: Drawing Survival Curves Using ggplot2](https://rdrr.io/cran/survminer/man/ggsurvplot.html)

https://cran.r-project.org/web/packages/ggfortify/vignettes/basics.html

In [ ]:
#define age line
AgeLine <- 65

## Primary analysis
- Positive Patients swabs after initital positive
- Censored after FIRST stop rule

In [ ]:
data_TTE <- SwabData_positive_censored_TTE_Primary

titles1 = "Time to viral clearance"

titles_list = c("Kaplan Meier Analysis",
               "Kaplan Meier ~ Sex",
               "Kaplan Meier ~ Age",
               "Kaplan Meier ~ Age + Sex")


fileNameType <- "PRIMARY_ALL"


subtitles <-"All COVID19 positive cases: Censored at stop rule"

### Cox Proportional Hazards Model

[ggforest: Forest Plot for Cox Proportional Hazards Model](https://rdrr.io/cran/survminer/man/ggforest.html)

[ggadjustedcurves: Adjusted Survival Curves for Cox Proportional Hazards Model ](https://rdrr.io/cran/survminer/man/ggadjustedcurves.html)

In [ ]:
titles <- "Adjusted Survival Curves for Cox Proportional Hazards Model"


In [ ]:
count(data_TTE$Status_Num)

In [ ]:
data_TTE_AGE <- mutate(data_TTE,
                        AgeFactor = ifelse((Age < AgeLine), 
                                           paste("Under", toString(AgeLine)),
                                           paste(toString(AgeLine), "and over")),
                       
                        AgeFactor = factor(AgeFactor),
                       
                        Sex = factor(Sex,
                                     labels=c("F","M")),
                       )


#Build the model
cox <- coxph(Surv(DaysToNeg, Status_Num) ~ AgeFactor + Sex, 
             data = data_TTE_AGE)



cox_sum <- summary(cox)

cox_sum

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COXForrest.png", sep = "")


#forest plot
p <- ggforest(cox,
             data = data_TTE_AGE,
             fontsize = 0.5)


ggsave(file = fileName, p)

p

In [ ]:
#https://rdrr.io/cran/survminer/man/ggadjustedcurves.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COX.png", sep = "")


#draw the plot
p <- ggadjustedcurves(cox,
                      data = data_TTE_AGE,
                      
                      #title and axis
                      title = titles1,
                      subtitle = subtitles,
                      xlab = "Day since first positive test", 
                      ylab = "% positive", 
                      
                      # Add confidence Intervals
                      conf.int = TRUE,
                      
                      # Change legends
                      legend = "none"
                     )

p

ggsave(file = fileName, p)

### Kaplan Meier Analysis

In [ ]:
km_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ 1, 
                  data = data_TTE)

km_fit

"*****************************"
km_fit_sum <- summary(km_fit, times = c(1,2*(1:20)))

time <- km_fit_sum$time
events <- km_fit_sum$n.event

kmsum_df <- data.frame(time,
                      events)

"Neg status achieved in the first 10 days"
sum(kmsum_df[kmsum_df$time <=10, 'events'])

"Neg status achieved in the first 14 days"
sum(kmsum_df[kmsum_df$time <=14, 'events'])
## Call: survfit(formula = Surv(time, status) ~ 1, data = veteran)

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC.png", sep = "")

titles = titles_list[1]


all <- ggsurvplot(km_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                #title = titles,
                #subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                #legend
                legend = c(0.85, 0.91),
                legend.title = "",
                legend.labs = c("All patients"),
                legend.fontsize = 2,
                
                # Add confidence Intervals
                conf.int = TRUE,
                
                
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                
                
                # Color 
                ggtheme = theme_bw() # Change ggplot2 theme
               
               
               )

# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

all$table <- all$table +
  theme(plot.title = element_text(size = 10))


all$plot <- all$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(all))


KM_all_all <- all

KM_all_all

[Compute P-value Comparing Survival Curves](https://rpkgs.datanovia.com/survminer/reference/surv_pvalue.html)

In [ ]:
#create the model
km_fit_Sex <- survfit(Surv(DaysToNeg, Status_Num) ~ Sex, 
                  data = data_TTE)

km_fit_Sex

"*****************************"
km_fit_sum <- summary(km_fit_Sex, times = c(1,2*(1:20)))

time <- km_fit_sum$time
events <- km_fit_sum$n.event

kmsum_df <- data.frame(time,
                      events)

"Neg status achieved in the first 10 days"
sum(kmsum_df[kmsum_df$time <=10, 'events'])

"Neg status achieved in the first 14 days"
sum(kmsum_df[kmsum_df$time <=14, 'events'])
## Call: survfit(formula = Surv(time, status) ~ 1, data = veteran)

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEX.png", sep = "")


km_fit_Sex <- survfit(Surv(DaysToNeg, Status_Num) ~ Sex, 
                  data = data_TTE)


titles = titles_list[2]


sex <- ggsurvplot(km_fit_Sex,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                #title = titles,
                #subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Sex",
                legend.labs = c("Female", "Male"),
                
                # Add p-value and tervals
                pval = round(surv_pvalue(km_fit_Sex)$pval, 3), 
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid3", 
                            "skyblue3"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

sex$table <- sex$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#second row is sex
row <- 2

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )



sex$plot <- sex$plot +

#change size of legend
theme(legend.text = element_text(size = 10)) +


#add annotation
annotate(geom="text", 
         x=40,
         y=1, 
         label=CoxHazard,
         color="grey20")



ggsave(file = fileName, print(sex))


KM_all_sex <- sex

KM_all_sex 


In [ ]:
#create the model
km_AG_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ AgeFactor, 
                     data = data_TTE_AGE)

km_AG_fit

"*****************************"
km_fit_sum <- summary(km_AG_fit, times = c(1,2*(1:20)))

time <- km_fit_sum$time
events <- km_fit_sum$n.event

kmsum_df <- data.frame(time,
                      events)

"Neg status achieved in the first 10 days"
sum(kmsum_df[kmsum_df$time <=10, 'events'])

"Neg status achieved in the first 14 days"
sum(kmsum_df[kmsum_df$time <=14, 'events'])
## Call: survfit(formula = Surv(time, status) ~ 1, data = veteran)

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_AGE.png", sep = "")




#plot the curves

titles = titles_list[3]

age <- ggsurvplot(km_AG_fit,
                
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                #title = titles,
                #subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Age group",
                legend.labs = c(paste(toString(AgeLine), "and over"),
                                paste("Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval =  round(surv_pvalue(km_AG_fit)$pval, 3),
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("#5ab4ac",
                           "#d8b365"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )


age$table <- age$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#first row is age
row <- 1

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )


age$plot <- age$plot +

#change legend size
theme(legend.text = element_text(size = 10))+


#add annotation
annotate(geom="text", 
         x=40,
         y=1, 
         label=CoxHazard,
         color="grey20")




ggsave(file = fileName, print(age))


KM_all_age <- age
KM_all_age

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEXAGE.png", sep = "")

#create the model
#https://rdrr.io/cran/survminer/man/surv_fit.html
km_AG_fit <- surv_fit(Surv(DaysToNeg, Status_Num) ~ Sex + AgeFactor, 
                     data = data_TTE_AGE)



titles = titles_list[4]

agesex <- ggsurvplot(km_AG_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #size of plot
                surv.plot.height = 2,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.8, 0.8),
                legend.title = "Age group and Sex",
                legend.labs = c(paste("Female: ", toString(AgeLine), "and over"),
                                paste("Female: Under", toString(AgeLine)),
                               paste("Male: ", toString(AgeLine), "and over"),
                                paste("Male: Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval = TRUE,
               # conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                               
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                tables.y.text.col = FALSE,
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid4",
                           "orchid1",
                           "skyblue4",
                           "skyblue1"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



agesex$table <- agesex$table +
  theme(plot.title = element_text(size = 10))


agesex$plot <- agesex$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(agesex))


KM_all_agesex <- agesex
KM_all_agesex

In [ ]:
#ref: https://rdrr.io/cran/survminer/man/arrange_ggsurvplots.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_JOINT.png", sep = "")


all$plot <- all$plot +

#change titles
labs(
    title = "A", #title
     subtitle = NULL 
    
)  
 

age$plot <- age$plot +

#change titles
labs(
    title = "B", #title
     subtitle = NULL 
    
)  
 

sex$plot <- sex$plot +

#change titles
labs(
    title = "C", #title
     subtitle = NULL 
    
)  


agesex$plot <- agesex$plot +

#change titles
labs(
    title = "D", #title
     subtitle = NULL 
    
)  
 
 


splots <-list (all,
               age,
              sex,
              agesex)


p <- arrange_ggsurvplots(splots, 
                    print = TRUE,
                    ncol = 2, 
                    nrow = 2, 
                    title = paste(titles1,
                                 " (", subtitles, ")",
                                 sep = ""),
                    #surv.plot.height = 0.8,
                    risk.table.height = 0.3
                   
                   )




ggsave(file = fileName, 
       p,
      height = 10 , 
       width = 10 )

p

************************
## Sensitivity analysis: Patients that had a final neg status
- Positive Patients swabs after initital positive
- Censored after FIRST stop rule

In [ ]:
data_TTE <- SwabData_positive_endNeg_Censored_TTE

titles1 = "Time to viral clearance"

titles_list = c("Kaplan Meier Analysis",
               "Kaplan Meier ~ Sex",
               "Kaplan Meier ~ Age",
               "Kaplan Meier ~ Age + Sex")


fileNameType <- "NEG_STATUS_MAINRULE"


subtitles <-"COVID19 positive cases that achieved a negative status: Censored at clinical stop rule"

### Cox Proportional Hazards Model

[ggforest: Forest Plot for Cox Proportional Hazards Model](https://rdrr.io/cran/survminer/man/ggforest.html)

[ggadjustedcurves: Adjusted Survival Curves for Cox Proportional Hazards Model ](https://rdrr.io/cran/survminer/man/ggadjustedcurves.html)

In [ ]:
titles <- "Adjusted Survival Curves for Cox Proportional Hazards Model"


In [ ]:
count(data_TTE$Status_Num)

In [ ]:
data_TTE_AGE <- mutate(data_TTE,
                        AgeFactor = ifelse((Age < AgeLine), 
                                           paste("Under", toString(AgeLine)),
                                           paste(toString(AgeLine), "and over")),
                       
                        AgeFactor = factor(AgeFactor),
                       
                        Sex = factor(Sex,
                                     labels=c("F","M")),
                       )


#Build the model
cox <- coxph(Surv(DaysToNeg, Status_Num) ~ AgeFactor + Sex, 
             data = data_TTE_AGE)



cox_sum <- summary(cox)



In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COXForrest.png", sep = "")


#forest plot
p <- ggforest(cox,
             data = data_TTE_AGE,
             fontsize = 0.5)


ggsave(file = fileName, p)

#p

In [ ]:
#https://rdrr.io/cran/survminer/man/ggadjustedcurves.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COX.png", sep = "")


#draw the plot
p <- ggadjustedcurves(cox,
                      data = data_TTE_AGE,
                      
                      #title and axis
                      title = titles1,
                      subtitle = subtitles,
                      xlab = "Day since first positive test", 
                      ylab = "% positive", 
                      
                      # Add confidence Intervals
                      conf.int = TRUE,
                      
                      # Change legends
                      legend = "none"
                     )

p

ggsave(file = fileName, p)

### Kaplan Meier Analysis

In [ ]:
km_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ 1, 
                  data = data_TTE)

km_fit

"*****************************"
km_fit_sum <- summary(km_fit, times = c(1,2*(1:20)))

time <- km_fit_sum$time
events <- km_fit_sum$n.event

kmsum_df <- data.frame(time,
                      events)

"Neg status achieved in the first 10 days"
sum(kmsum_df[kmsum_df$time <=10, 'events'])

"Neg status achieved in the first 14 days"
sum(kmsum_df[kmsum_df$time <=14, 'events'])

## Call: survfit(formula = Surv(time, status) ~ 1, data = veteran)

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC.png", sep = "")

titles = titles_list[1]


all <- ggsurvplot(km_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                #title = titles,
                #subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                #legend
                legend = c(0.85, 0.91),
                legend.title = "",
                legend.labs = c("All patients"),
                legend.fontsize = 2,
                
                # Add confidence Intervals
                conf.int = TRUE,
                
                
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                
                
                # Color 
                ggtheme = theme_bw() # Change ggplot2 theme
               
               
               )

# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

all$table <- all$table +
  theme(plot.title = element_text(size = 10))


all$plot <- all$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(all))


KM_FinalNegMainRule_all <- all

KM_FinalNegMainRule_all

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEX.png", sep = "")


km_fit_Sex <- survfit(Surv(DaysToNeg, Status_Num) ~ Sex, 
                  data = data_TTE)


titles = titles_list[2]


sex <- ggsurvplot(km_fit_Sex,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Sex",
                legend.labs = c("Female", "Male"),
                
                # Add p-value and tervals
                pval = TRUE,
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid3", 
                            "skyblue3"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

sex$table <- sex$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#second row is sex
row <- 2

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )



sex$plot <- sex$plot +

#change size of legend
theme(legend.text = element_text(size = 10)) +


#add annotation
annotate(geom="text", 
         x=30,
         y=1, 
         label=CoxHazard,
         color="grey20")



ggsave(file = fileName, print(sex))


KM_FinalNegMainRule_sex <- sex

KM_FinalNegMainRule_sex 


In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_AGE.png", sep = "")


#create the model
km_AG_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ AgeFactor, 
                     data = data_TTE_AGE)


#plot the curves

titles = titles_list[3]

age <- ggsurvplot(km_AG_fit,
                
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Age group",
                legend.labs = c(paste(toString(AgeLine), "and over"),
                                paste("Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval = TRUE,
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("#5ab4ac",
                           "#d8b365"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )


age$table <- age$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#first row is age
row <- 1

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )


age$plot <- age$plot +

#change legend size
theme(legend.text = element_text(size = 10))+


#add annotation
annotate(geom="text", 
         x=30,
         y=1, 
         label=CoxHazard,
         color="grey20")




ggsave(file = fileName, print(age))


KM_FinalNegMainRule_age <- age
KM_FinalNegMainRule_age

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEXAGE.png", sep = "")

#create the model
#https://rdrr.io/cran/survminer/man/surv_fit.html
km_AG_fit <- surv_fit(Surv(DaysToNeg, Status_Num) ~ Sex + AgeFactor, 
                     data = data_TTE_AGE)



titles = titles_list[4]

agesex <- ggsurvplot(km_AG_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #size of plot
                surv.plot.height = 2,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.8, 0.8),
                legend.title = "Age group and Sex",
                legend.labs = c(paste("Female: ", toString(AgeLine), "and over"),
                                paste("Female: Under", toString(AgeLine)),
                               paste("Male: ", toString(AgeLine), "and over"),
                                paste("Male: Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval = TRUE,
               # conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                               
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                tables.y.text.col = FALSE,
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid4",
                           "orchid1",
                           "skyblue4",
                           "skyblue1"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



agesex$table <- agesex$table +
  theme(plot.title = element_text(size = 10))


agesex$plot <- agesex$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(agesex))


KM_FinalNegMainRule_agesex <- agesex
KM_FinalNegMainRule_agesex

In [ ]:
#ref: https://rdrr.io/cran/survminer/man/arrange_ggsurvplots.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_JOINT.png", sep = "")


all$plot <- all$plot +

#change titles
labs(
    title = "A", #title
     subtitle = NULL 
    
)  
 

age$plot <- age$plot +

#change titles
labs(
    title = "B", #title
     subtitle = NULL 
    
)  
 

sex$plot <- sex$plot +

#change titles
labs(
    title = "C", #title
     subtitle = NULL 
    
)  


agesex$plot <- agesex$plot +

#change titles
labs(
    title = "D", #title
     subtitle = NULL 
    
)  
 
 


splots <-list (all,
               age,
              sex,
              agesex)


p <- arrange_ggsurvplots(splots, 
                    print = TRUE,
                    ncol = 2, 
                    nrow = 2, 
                    title = paste(titles1,
                                 " (", subtitles, ")",
                                 sep = ""),
                    #surv.plot.height = 0.8,
                    risk.table.height = 0.3
                   
                   )




ggsave(file = fileName, 
       p,
      height = 10 , 
       width = 10 )

p

************************
## Sensitivity analysis: Patients that had a final neg status according to "new rule"
- Positive Patients swabs after initital positive
- Censored after FIRST stop rule

In [ ]:
data_TTE <- SwabData_positive_endNeg_Censored_NEWRULE_TTE

titles1 = "Time to viral clearance"

titles_list = c("Kaplan Meier Analysis",
               "Kaplan Meier ~ Sex",
               "Kaplan Meier ~ Age",
               "Kaplan Meier ~ Age + Sex")


fileNameType <- "NEG_STATUS_NEWRULE"


subtitles <-"All COVID19 positive cases that achieved a negative status: Censored at alternate stop rule"

### Cox Proportional Hazards Model

[ggforest: Forest Plot for Cox Proportional Hazards Model](https://rdrr.io/cran/survminer/man/ggforest.html)

[ggadjustedcurves: Adjusted Survival Curves for Cox Proportional Hazards Model ](https://rdrr.io/cran/survminer/man/ggadjustedcurves.html)

In [ ]:
titles <- "Adjusted Survival Curves for Cox Proportional Hazards Model"


In [ ]:
count(data_TTE$Status_Num)

In [ ]:
data_TTE_AGE <- mutate(data_TTE,
                        AgeFactor = ifelse((Age < AgeLine), 
                                           paste("Under", toString(AgeLine)),
                                           paste(toString(AgeLine), "and over")),
                       
                       
                       
                        AgeFactor = factor(AgeFactor),
                       
                        Sex = factor(Sex,
                                     labels=c("F","M")),
                       )


#Build the model
cox <- coxph(Surv(DaysToNeg, Status_Num) ~ AgeFactor + Sex, 
             data = data_TTE_AGE)



cox_sum <- summary(cox)



In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COXForrest.png", sep = "")


#forest plot
p <- ggforest(cox,
             data = data_TTE_AGE,
             fontsize = 0.5)


ggsave(file = fileName, p)

#p

In [ ]:
#https://rdrr.io/cran/survminer/man/ggadjustedcurves.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COX.png", sep = "")


#draw the plot
p <- ggadjustedcurves(cox,
                      data = data_TTE_AGE,
                      
                      #title and axis
                      title = titles1,
                      subtitle = subtitles,
                      xlab = "Day since first positive test", 
                      ylab = "% positive", 
                      
                      # Add confidence Intervals
                      conf.int = TRUE,
                      
                      # Change legends
                      legend = "none"
                     )

p

ggsave(file = fileName, p)

### Kaplan Meier Analysis

In [ ]:
km_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ 1, 
                  data = data_TTE)

km_fit

"*****************************"
km_fit_sum <- summary(km_fit, times = c(1,2*(1:20)))

time <- km_fit_sum$time
events <- km_fit_sum$n.event

kmsum_df <- data.frame(time,
                      events)

"Neg status achieved in the first 10 days"
sum(kmsum_df[kmsum_df$time <=10, 'events'])

"Neg status achieved in the first 14 days"
sum(kmsum_df[kmsum_df$time <=14, 'events'])

## Call: survfit(formula = Surv(time, status) ~ 1, data = veteran)

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC.png", sep = "")

titles = titles_list[1]


all <- ggsurvplot(km_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                #legend
                legend = c(0.9, 0.91),
                legend.title = "",
                legend.labs = c("All patients"),
                legend.fontsize = 2,
                
                # Add confidence Intervals
                conf.int = TRUE,
                
                
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                
                
                # Color 
                ggtheme = theme_bw() # Change ggplot2 theme
               
               
               )

# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

all$table <- all$table +
  theme(plot.title = element_text(size = 10))


all$plot <- all$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(all))


KM_FinalNegNEWRule_all <- all

KM_FinalNegNEWRule_all

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEX.png", sep = "")


km_fit_Sex <- survfit(Surv(DaysToNeg, Status_Num) ~ Sex, 
                  data = data_TTE)


titles = titles_list[2]


sex <- ggsurvplot(km_fit_Sex,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Sex",
                legend.labs = c("Female", "Male"),
                
                # Add p-value and tervals
                pval = TRUE,
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid3", 
                            "skyblue3"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

sex$table <- sex$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#second row is sex
row <- 2

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )



sex$plot <- sex$plot +

#change size of legend
theme(legend.text = element_text(size = 10)) +


#add annotation
annotate(geom="text", 
         x=40,
         y=1, 
         label=CoxHazard,
         color="grey20")



ggsave(file = fileName, print(sex))


KM_FinalNegNEWRule_sex <- sex

KM_FinalNegNEWRule_sex 


In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_AGE.png", sep = "")


#create the model
km_AG_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ AgeFactor, 
                     data = data_TTE_AGE)


#plot the curves

titles = titles_list[3]

age <- ggsurvplot(km_AG_fit,
                
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Age group",
                legend.labs = c(paste(toString(AgeLine), "and over"),
                                paste("Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval = TRUE,
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("#5ab4ac",
                           "#d8b365"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )


age$table <- age$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#first row is age
row <- 1

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )


age$plot <- age$plot +

#change legend size
theme(legend.text = element_text(size = 10))+


#add annotation
annotate(geom="text", 
         x=40,
         y=1, 
         label=CoxHazard,
         color="grey20")




ggsave(file = fileName, print(age))


KM_FinalNegNEWRule_age <- age
KM_FinalNegNEWRule_age

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEXAGE.png", sep = "")

#create the model
#https://rdrr.io/cran/survminer/man/surv_fit.html
km_AG_fit <- surv_fit(Surv(DaysToNeg, Status_Num) ~ Sex + AgeFactor, 
                     data = data_TTE_AGE)



titles = titles_list[4]

agesex <- ggsurvplot(km_AG_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #size of plot
                surv.plot.height = 2,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.8, 0.8),
                legend.title = "Age group and Sex",
                legend.labs = c(paste("Female: ", toString(AgeLine), "and over"),
                                paste("Female: Under", toString(AgeLine)),
                               paste("Male: ", toString(AgeLine), "and over"),
                                paste("Male: Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval = TRUE,
               # conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                               
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                tables.y.text.col = FALSE,
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid4",
                           "orchid1",
                           "skyblue4",
                           "skyblue1"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



agesex$table <- agesex$table +
  theme(plot.title = element_text(size = 10))


agesex$plot <- agesex$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(agesex))


KM_FinalNegNEWRule_agesex <- agesex
KM_FinalNegNEWRule_agesex

In [ ]:
#ref: https://rdrr.io/cran/survminer/man/arrange_ggsurvplots.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_JOINT.png", sep = "")


all$plot <- all$plot +

#change titles
labs(
    title = "A", #title
     subtitle = NULL 
    
)  
 

age$plot <- age$plot +

#change titles
labs(
    title = "B", #title
     subtitle = NULL 
    
)  
 

sex$plot <- sex$plot +

#change titles
labs(
    title = "C", #title
     subtitle = NULL 
    
)  


agesex$plot <- agesex$plot +

#change titles
labs(
    title = "D", #title
     subtitle = NULL 
    
)  
 
 


splots <-list (all,
               age,
              sex,
              agesex)


p <- arrange_ggsurvplots(splots, 
                    print = TRUE,
                    ncol = 2, 
                    nrow = 2, 
                    title = paste(titles1,
                                 " (", subtitles, ")",
                                 sep = ""),
                    #surv.plot.height = 0.8,
                    risk.table.height = 0.3
                   
                   )




ggsave(file = fileName, 
       p,
      height = 10 , 
       width = 10 )

p

************************
## Sensitivity analysis: All patients - UNCENSORED at stop rule
- Positive Patients swabs after initital positive
- NOT censored after FIRST stop rule

In [ ]:
data_TTE <- SwabData_positive_UNcensored_TTE_Sensitivity

titles1 = "Time to viral clearance"

titles_list = c("Kaplan Meier Analysis",
               "Kaplan Meier ~ Sex",
               "Kaplan Meier ~ Age",
               "Kaplan Meier ~ Age + Sex")


fileNameType <- "UNCENSORED_ALL"


subtitles <-"All COVID19 positive cases: Not censored at clinical stop rule"

### Cox Proportional Hazards Model

[ggforest: Forest Plot for Cox Proportional Hazards Model](https://rdrr.io/cran/survminer/man/ggforest.html)

[ggadjustedcurves: Adjusted Survival Curves for Cox Proportional Hazards Model ](https://rdrr.io/cran/survminer/man/ggadjustedcurves.html)

In [ ]:
titles <- "Adjusted Survival Curves for Cox Proportional Hazards Model"


In [ ]:
count(data_TTE$Status_Num)

In [ ]:
data_TTE_AGE <- mutate(data_TTE,
                        AgeFactor = ifelse((Age < AgeLine), 
                                           paste("Under", toString(AgeLine)),
                                           paste(toString(AgeLine), "and over")),
                       
                        AgeFactor = factor(AgeFactor),
                       
                        Sex = factor(Sex,
                                     labels=c("F","M")),
                       )


#Build the model
cox <- coxph(Surv(DaysToNeg, Status_Num) ~ AgeFactor + Sex, 
             data = data_TTE_AGE)



cox_sum <- summary(cox)



In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COXForrest.png", sep = "")


#forest plot
p <- ggforest(cox,
             data = data_TTE_AGE,
             fontsize = 0.5)


ggsave(file = fileName, p)

#p

In [ ]:
#https://rdrr.io/cran/survminer/man/ggadjustedcurves.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COX.png", sep = "")


#draw the plot
p <- ggadjustedcurves(cox,
                      data = data_TTE_AGE,
                      
                      #title and axis
                      title = titles1,
                      subtitle = subtitles,
                      xlab = "Day since first positive test", 
                      ylab = "% positive", 
                      
                      # Add confidence Intervals
                      conf.int = TRUE,
                      
                      # Change legends
                      legend = "none"
                     )

p

ggsave(file = fileName, p)

### Kaplan Meier Analysis

In [ ]:
km_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ 1, 
                  data = data_TTE)


summary(km_fit, times = c(1,5,10,15,5*(1:10)))

## Call: survfit(formula = Surv(time, status) ~ 1, data = veteran)

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC.png", sep = "")

titles = titles_list[1]


all <- ggsurvplot(km_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                #legend
                legend = c(0.9, 0.91),
                legend.title = "",
                legend.labs = c("All patients"),
                legend.fontsize = 2,
                
                # Add confidence Intervals
                conf.int = TRUE,
                
                
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                
                
                # Color 
                ggtheme = theme_bw() # Change ggplot2 theme
               
               
               )

# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

all$table <- all$table +
  theme(plot.title = element_text(size = 10))


all$plot <- all$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(all))


KM_UNCENS_all_all <- all

KM_UNCENS_all_all

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEX.png", sep = "")


km_fit_Sex <- survfit(Surv(DaysToNeg, Status_Num) ~ Sex, 
                  data = data_TTE)


titles = titles_list[2]


sex <- ggsurvplot(km_fit_Sex,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Sex",
                legend.labs = c("Female", "Male"),
                
                # Add p-value and tervals
                pval = TRUE,
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid3", 
                            "skyblue3"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

sex$table <- sex$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#second row is sex
row <- 2

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )



sex$plot <- sex$plot +

#change size of legend
theme(legend.text = element_text(size = 10)) +


#add annotation
annotate(geom="text", 
         x=40,
         y=1, 
         label=CoxHazard,
         color="grey20")



ggsave(file = fileName, print(sex))


KM_UNCENS_all_sex <- sex

KM_UNCENS_all_sex 


In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_AGE.png", sep = "")


#create the model
km_AG_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ AgeFactor, 
                     data = data_TTE_AGE)


#plot the curves

titles = titles_list[3]

age <- ggsurvplot(km_AG_fit,
                
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Age group",
                legend.labs = c(paste(toString(AgeLine), "and over"),
                                paste("Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval = TRUE,
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("#5ab4ac",
                           "#d8b365"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )


age$table <- age$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#first row is age
row <- 1

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )


age$plot <- age$plot +

#change legend size
theme(legend.text = element_text(size = 10))+


#add annotation
annotate(geom="text", 
         x=40,
         y=1, 
         label=CoxHazard,
         color="grey20")




ggsave(file = fileName, print(age))


KM_UNCENS_all_age <- age
KM_UNCENS_all_age

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEXAGE.png", sep = "")

#create the model
#https://rdrr.io/cran/survminer/man/surv_fit.html
km_AG_fit <- surv_fit(Surv(DaysToNeg, Status_Num) ~ Sex + AgeFactor, 
                     data = data_TTE_AGE)



titles = titles_list[4]

agesex <- ggsurvplot(km_AG_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #size of plot
                surv.plot.height = 2,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.8, 0.8),
                legend.title = "Age group and Sex",
                legend.labs = c(paste("Female: ", toString(AgeLine), "and over"),
                                paste("Female: Under", toString(AgeLine)),
                               paste("Male: ", toString(AgeLine), "and over"),
                                paste("Male: Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval = TRUE,
               # conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                               
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                tables.y.text.col = FALSE,
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid4",
                           "orchid1",
                           "skyblue4",
                           "skyblue1"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



agesex$table <- agesex$table +
  theme(plot.title = element_text(size = 10))


agesex$plot <- agesex$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(agesex))


KM_UNCENS_all_agesex <- agesex
KM_UNCENS_all_agesex

In [ ]:
#ref: https://rdrr.io/cran/survminer/man/arrange_ggsurvplots.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_JOINT.png", sep = "")


all$plot <- all$plot +

#change titles
labs(
    title = "A", #title
     subtitle = NULL 
    
)  
 

age$plot <- age$plot +

#change titles
labs(
    title = "B", #title
     subtitle = NULL 
    
)  
 

sex$plot <- sex$plot +

#change titles
labs(
    title = "C", #title
     subtitle = NULL 
    
)  


agesex$plot <- agesex$plot +

#change titles
labs(
    title = "D", #title
     subtitle = NULL 
    
)  
 
 


splots <-list (all,
               age,
              sex,
              agesex)


p <- arrange_ggsurvplots(splots, 
                    print = TRUE,
                    ncol = 2, 
                    nrow = 2, 
                    title = paste(titles1,
                                 " (", subtitles, ")",
                                 sep = ""),
                    #surv.plot.height = 0.8,
                    risk.table.height = 0.3
                   
                   )




ggsave(file = fileName, 
       p,
      height = 10 , 
       width = 10 )

p

************************
## Sensitivity analysis: All patients that achieved neg status - UNCENSORED at first stop rule
- Positive Patients swabs after initital positive
- NOT censored after FIRST stop rule

In [ ]:
data_TTE <- SwabData_positive_endNeg_UNCensored_TTE

titles1 = "Time to viral clearance"

titles_list = c("Kaplan Meier Analysis",
               "Kaplan Meier ~ Sex",
               "Kaplan Meier ~ Age",
               "Kaplan Meier ~ Age + Sex")


fileNameType <- "UNCENSORED_NEG_STATUS_MAINRULE"


subtitles <-"All COVID19 positive cases that achieved a negative status: Not censored at clinical stop rule"

### Cox Proportional Hazards Model

[ggforest: Forest Plot for Cox Proportional Hazards Model](https://rdrr.io/cran/survminer/man/ggforest.html)

[ggadjustedcurves: Adjusted Survival Curves for Cox Proportional Hazards Model ](https://rdrr.io/cran/survminer/man/ggadjustedcurves.html)

In [ ]:
titles <- "Adjusted Survival Curves for Cox Proportional Hazards Model"


In [ ]:
count(data_TTE$Status_Num)

In [ ]:
data_TTE_AGE <- mutate(data_TTE,
                        AgeFactor = ifelse((Age < AgeLine), 
                                           paste("Under", toString(AgeLine)),
                                           paste(toString(AgeLine), "and over")),
                       
                        AgeFactor = factor(AgeFactor),
                       
                        Sex = factor(Sex,
                                     labels=c("F","M")),
                       )


#Build the model
cox <- coxph(Surv(DaysToNeg, Status_Num) ~ AgeFactor + Sex, 
             data = data_TTE_AGE)



cox_sum <- summary(cox)



In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COXForrest.png", sep = "")


#forest plot
p <- ggforest(cox,
             data = data_TTE_AGE,
             fontsize = 0.5)


ggsave(file = fileName, p)

#p

In [ ]:
#https://rdrr.io/cran/survminer/man/ggadjustedcurves.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COX.png", sep = "")


#draw the plot
p <- ggadjustedcurves(cox,
                      data = data_TTE_AGE,
                      
                      #title and axis
                      title = titles1,
                      subtitle = subtitles,
                      xlab = "Day since first positive test", 
                      ylab = "% positive", 
                      
                      # Add confidence Intervals
                      conf.int = TRUE,
                      
                      # Change legends
                      legend = "none"
                     )

p

ggsave(file = fileName, p)

### Kaplan Meier Analysis

In [ ]:
km_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ 1, 
                  data = data_TTE)


summary(km_fit, times = c(1,5,10,15,5*(1:10)))

## Call: survfit(formula = Surv(time, status) ~ 1, data = veteran)

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC.png", sep = "")

titles = titles_list[1]


all <- ggsurvplot(km_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                #legend
                legend = c(0.9, 0.91),
                legend.title = "",
                legend.labs = c("All patients"),
                legend.fontsize = 2,
                
                # Add confidence Intervals
                conf.int = TRUE,
                
                
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                
                
                # Color 
                ggtheme = theme_bw() # Change ggplot2 theme
               
               
               )

# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

all$table <- all$table +
  theme(plot.title = element_text(size = 10))


all$plot <- all$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(all))


KM_UNCENS_FinalNegMainRule_all <- all

KM_UNCENS_FinalNegMainRule_all

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEX.png", sep = "")


km_fit_Sex <- survfit(Surv(DaysToNeg, Status_Num) ~ Sex, 
                  data = data_TTE)


titles = titles_list[2]


sex <- ggsurvplot(km_fit_Sex,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Sex",
                legend.labs = c("Female", "Male"),
                
                # Add p-value and tervals
                pval = TRUE,
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid3", 
                            "skyblue3"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

sex$table <- sex$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#second row is sex
row <- 2

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )



sex$plot <- sex$plot +

#change size of legend
theme(legend.text = element_text(size = 10)) +


#add annotation
annotate(geom="text", 
         x=30,
         y=1, 
         label=CoxHazard,
         color="grey20")



ggsave(file = fileName, print(sex))


KM_UNCENS_FinalNegMainRule_sex <- sex

KM_UNCENS_FinalNegMainRule_sex 


In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_AGE.png", sep = "")


#create the model
km_AG_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ AgeFactor, 
                     data = data_TTE_AGE)


#plot the curves

titles = titles_list[3]

age <- ggsurvplot(km_AG_fit,
                
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Age group",
                legend.labs = c(paste(toString(AgeLine), "and over"),
                                paste("Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval = TRUE,
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("#5ab4ac",
                           "#d8b365"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )


age$table <- age$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#first row is age
row <- 1

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )


age$plot <- age$plot +

#change legend size
theme(legend.text = element_text(size = 10))+


#add annotation
annotate(geom="text", 
         x=30,
         y=1, 
         label=CoxHazard,
         color="grey20")




ggsave(file = fileName, print(age))


KM_UNCENS_FinalNegMainRule_age <- age
KM_UNCENS_FinalNegMainRule_age

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEXAGE.png", sep = "")

#create the model
#https://rdrr.io/cran/survminer/man/surv_fit.html
km_AG_fit <- surv_fit(Surv(DaysToNeg, Status_Num) ~ Sex + AgeFactor, 
                     data = data_TTE_AGE)



titles = titles_list[4]

agesex <- ggsurvplot(km_AG_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #size of plot
                surv.plot.height = 2,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.8, 0.8),
                legend.title = "Age group and Sex",
                legend.labs = c(paste("Female: ", toString(AgeLine), "and over"),
                                paste("Female: Under", toString(AgeLine)),
                               paste("Male: ", toString(AgeLine), "and over"),
                                paste("Male: Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval = TRUE,
               # conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                               
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                tables.y.text.col = FALSE,
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid4",
                           "orchid1",
                           "skyblue4",
                           "skyblue1"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



agesex$table <- agesex$table +
  theme(plot.title = element_text(size = 10))


agesex$plot <- agesex$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(agesex))


KM_UNCENS_FinalNegMainRule_agesex <- agesex
KM_UNCENS_FinalNegMainRule_agesex

In [ ]:
#ref: https://rdrr.io/cran/survminer/man/arrange_ggsurvplots.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_JOINT.png", sep = "")


all$plot <- all$plot +

#change titles
labs(
    title = "A", #title
     subtitle = NULL 
    
)  
 

age$plot <- age$plot +

#change titles
labs(
    title = "B", #title
     subtitle = NULL 
    
)  
 

sex$plot <- sex$plot +

#change titles
labs(
    title = "C", #title
     subtitle = NULL 
    
)  


agesex$plot <- agesex$plot +

#change titles
labs(
    title = "D", #title
     subtitle = NULL 
    
)  
 
 


splots <-list (all,
               age,
              sex,
              agesex)


p <- arrange_ggsurvplots(splots, 
                    print = TRUE,
                    ncol = 2, 
                    nrow = 2, 
                    title = paste(titles1,
                                 " (", subtitles, ")",
                                 sep = ""),
                    #surv.plot.height = 0.8,
                    risk.table.height = 0.3
                   
                   )




ggsave(file = fileName, 
       p,
      height = 10 , 
       width = 10 )

p

************************
## Sensitivity analysis: All patients that achieved neg status alternative rule - UNCENSORED at first stop rule
- Positive Patients swabs after initital positive
- NOT censored after FIRST stop rule

In [ ]:
data_TTE <- SwabData_positive_endNeg_UNCensored_NEWRULE_TTE

titles1 = "Time to viral clearance"

titles_list = c("Kaplan Meier Analysis",
               "Kaplan Meier ~ Sex",
               "Kaplan Meier ~ Age",
               "Kaplan Meier ~ Age + Sex")


fileNameType <- "UNCENSORED_NEG_STATUS_NEWRULE"


subtitles <-"All COVID19 positive cases that achieved a negative status: Not censored at alternate stop rule"

### Cox Proportional Hazards Model

[ggforest: Forest Plot for Cox Proportional Hazards Model](https://rdrr.io/cran/survminer/man/ggforest.html)

[ggadjustedcurves: Adjusted Survival Curves for Cox Proportional Hazards Model ](https://rdrr.io/cran/survminer/man/ggadjustedcurves.html)

In [ ]:
titles <- "Adjusted Survival Curves for Cox Proportional Hazards Model"


In [ ]:
count(data_TTE$Status_Num)

In [ ]:
data_TTE_AGE <- mutate(data_TTE,
                        AgeFactor = ifelse((Age < AgeLine), 
                                           paste("Under", toString(AgeLine)),
                                           paste(toString(AgeLine), "and over")),
                       
                        AgeFactor = factor(AgeFactor),
                       
                        Sex = factor(Sex,
                                     labels=c("F","M")),
                       )


#Build the model
cox <- coxph(Surv(DaysToNeg, Status_Num) ~ AgeFactor + Sex, 
             data = data_TTE_AGE)



cox_sum <- summary(cox)



In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COXForrest.png", sep = "")


#forest plot
p <- ggforest(cox,
             data = data_TTE_AGE,
             fontsize = 0.5)


ggsave(file = fileName, p)

#p

In [ ]:
#https://rdrr.io/cran/survminer/man/ggadjustedcurves.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_COX.png", sep = "")


#draw the plot
p <- ggadjustedcurves(cox,
                      data = data_TTE_AGE,
                      
                      #title and axis
                      title = titles1,
                      subtitle = subtitles,
                      xlab = "Day since first positive test", 
                      ylab = "% positive", 
                      
                      # Add confidence Intervals
                      conf.int = TRUE,
                      
                      # Change legends
                      legend = "none"
                     )

p

ggsave(file = fileName, p)

### Kaplan Meier Analysis

In [ ]:
km_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ 1, 
                  data = data_TTE)


summary(km_fit, times = c(1,5,10,15,5*(1:10)))

## Call: survfit(formula = Surv(time, status) ~ 1, data = veteran)

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC.png", sep = "")

titles = titles_list[1]


all <- ggsurvplot(km_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                #legend
                legend = c(0.9, 0.91),
                legend.title = "",
                legend.labs = c("All patients"),
                legend.fontsize = 2,
                
                # Add confidence Intervals
                conf.int = TRUE,
                
                
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                
                
                # Color 
                ggtheme = theme_bw() # Change ggplot2 theme
               
               
               )

# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

all$table <- all$table +
  theme(plot.title = element_text(size = 10))


all$plot <- all$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(all))


KM_UNCENS_FinalNegNEWRule_all <- all

KM_UNCENS_FinalNegNEWRule_all

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEX.png", sep = "")


km_fit_Sex <- survfit(Surv(DaysToNeg, Status_Num) ~ Sex, 
                  data = data_TTE)


titles = titles_list[2]


sex <- ggsurvplot(km_fit_Sex,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Sex",
                legend.labs = c("Female", "Male"),
                
                # Add p-value and tervals
                pval = TRUE,
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid3", 
                            "skyblue3"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



# Customize risk table title using ggplot verbs
#ref: https://github.com/kassambara/survminer/issues/255

sex$table <- sex$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#second row is sex
row <- 2

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )



sex$plot <- sex$plot +

#change size of legend
theme(legend.text = element_text(size = 10)) +


#add annotation
annotate(geom="text", 
         x=40,
         y=1, 
         label=CoxHazard,
         color="grey20")



ggsave(file = fileName, print(sex))


KM_UNCENS_FinalNegNEWRule_sex <- sex

KM_UNCENS_FinalNegNEWRule_sex 


In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_AGE.png", sep = "")


#create the model
km_AG_fit <- survfit(Surv(DaysToNeg, Status_Num) ~ AgeFactor, 
                     data = data_TTE_AGE)


#plot the curves

titles = titles_list[3]

age <- ggsurvplot(km_AG_fit,
                
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.9, 0.90),
                legend.title = "Age group",
                legend.labs = c(paste(toString(AgeLine), "and over"),
                                paste("Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval = TRUE,
                conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("#5ab4ac",
                           "#d8b365"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )


age$table <- age$table +
  theme(plot.title = element_text(size = 10))




#create annotation of Cox Hazard
#first row is age
row <- 1

CoxHazard <- paste("HR: ",
                   round(as.numeric(cox_sum$conf.int[row,1]), 2),
                   " (95% CI ",
                   round(as.numeric(cox_sum$conf.int[row,3]), 2),
                   ", ",
                   round(as.numeric(cox_sum$conf.int[row,4]), 2),
                   ")",
                   sep = ""
                  )


age$plot <- age$plot +

#change legend size
theme(legend.text = element_text(size = 10))+


#add annotation
annotate(geom="text", 
         x=40,
         y=1, 
         label=CoxHazard,
         color="grey20")




ggsave(file = fileName, print(age))


KM_UNCENS_FinalNegNEWRule_age <- age
KM_UNCENS_FinalNegNEWRule_age

In [ ]:
fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_SEXAGE.png", sep = "")

#create the model
#https://rdrr.io/cran/survminer/man/surv_fit.html
km_AG_fit <- surv_fit(Surv(DaysToNeg, Status_Num) ~ Sex + AgeFactor, 
                     data = data_TTE_AGE)



titles = titles_list[4]

agesex <- ggsurvplot(km_AG_fit,
                surv.median.line = "hv", # Add medians survival
                
                censor = TRUE,
                
                #size of plot
                surv.plot.height = 2,
                
                #title and axis
                title = titles,
                subtitle = subtitles,
                xlab = "Day since first positive test", 
                ylab = "% positive", 
                
                # Change legends: title & labels
                legend = c(0.8, 0.8),
                legend.title = "Age group and Sex",
                legend.labs = c(paste("Female: ", toString(AgeLine), "and over"),
                                paste("Female: Under", toString(AgeLine)),
                               paste("Male: ", toString(AgeLine), "and over"),
                                paste("Male: Under", toString(AgeLine))),
                
                # Add p-value and tervals
                pval = TRUE,
               # conf.int = TRUE,
                
                # Add risk table
                risk.table = "nrisk_cumevents",
                risk.table.title = "Number at risk (cumulative negative status)",
                fontsize = 2.5,
                
                               
                #format tables
                tables.height = 0.2,
                tables.theme = theme_cleantable(),
                tables.y.text.col = FALSE,
                
                # Color palettes. Use custom color: c("#E7B800", "#2E9FDF"),
                # or brewer color (e.g.: "Dark2"), or ggsci color (e.g.: "jco")
                palette = c("orchid4",
                           "orchid1",
                           "skyblue4",
                           "skyblue1"),
                ggtheme = theme_bw() # Change ggplot2 theme
               
               )



agesex$table <- agesex$table +
  theme(plot.title = element_text(size = 10))


agesex$plot <- agesex$plot +
theme(legend.text = element_text(size = 10))



ggsave(file = fileName, print(agesex))


KM_UNCENS_FinalNegNEWRule_agesex <- agesex
KM_UNCENS_FinalNegNEWRule_agesex

In [ ]:
#ref: https://rdrr.io/cran/survminer/man/arrange_ggsurvplots.html

fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_JOINT.png", sep = "")


all$plot <- all$plot +

#change titles
labs(
    title = "A", #title
     subtitle = NULL 
    
)  
 

age$plot <- age$plot +

#change titles
labs(
    title = "B", #title
     subtitle = NULL 
    
)  
 

sex$plot <- sex$plot +

#change titles
labs(
    title = "C", #title
     subtitle = NULL 
    
)  


agesex$plot <- agesex$plot +

#change titles
labs(
    title = "D", #title
     subtitle = NULL 
    
)  
 
 


splots <-list (all,
               age,
              sex,
              agesex)


p <- arrange_ggsurvplots(splots, 
                    print = TRUE,
                    ncol = 2, 
                    nrow = 2, 
                    title = paste(titles1,
                                 " (", subtitles, ")",
                                 sep = ""),
                    #surv.plot.height = 0.8,
                    risk.table.height = 0.3
                   
                   )




ggsave(file = fileName, 
       p,
      height = 10 , 
       width = 10 )

p

In [ ]:
fileNameType = "ALL_Patients_PRIMARY"

#subtitles = "All positive patients v those that achieved a negative status"

titles = titles_list[1]


fileName <- paste("Outputs/Visuals/TTEAnalysis/KMC_", fileNameType,".png", sep = "")




KM_FinalNegMainRule_all$plot <- KM_FinalNegMainRule_all$plot +

#change titles
labs(title = "",
     #"A: Patients who achieved a negative status (2 negative swabs)", #title
     subtitle = NULL 
    
) 

KM_FinalNegMainRule_all

ggsave(file = fileName, print(KM_FinalNegMainRule_all))

In [ ]:
fileNameType = "NEGSTATUS_Patients_PRIMARY"

subtitles = "All positive patients v those that achieved a negative status"

titles = titles_list[1]



fileName <- paste("Outputs/Visuals/TTEAnalysis/KMC_", fileNameType,".png", sep = "")




KM_all_all$plot <- KM_all_all$plot +

#change titles
labs(title = "",
     #"B: All positive patients", #title
     subtitle = NULL 
    
) 



ggsave(file = fileName, print(KM_all_all))

In [ ]:
fileNameType = "ALL_Patients_PRIMARY~SEX"

subtitles = "All positive patients v those that achieved a negative status"

titles = titles_list[1]



fileName <- paste("Outputs/Visuals/TTEAnalysis/KMC_", fileNameType,".png", sep = "")




KM_all_sex$plot <- KM_all_sex$plot +

#change titles
labs(
    title = NULL, #title
    subtitle = NULL,
     caption = "HR = Hazard Ratio\nReference group:  males" 
    
)  +

theme(legend.position= c(0.85, 0.80))


ggsave(file = fileName, print(KM_all_sex))

In [ ]:
fileNameType = "ALL_Patients_PRIMARY~AGE"

subtitles = "All positive patients v those that achieved a negative status"

titles = titles_list[1]



fileName <- paste("Outputs/Visuals/TTEAnalysis/KMC_", fileNameType,".png", sep = "")




KM_all_age$plot <- KM_all_age$plot +

#change titles
labs(
    title = NULL, #title
    subtitle = NULL,
     caption = "HR = Hazard Ratio\nReference group:  65y and over" 
    
)  +

theme(legend.position= c(0.85, 0.82))


ggsave(file = fileName, print(KM_all_age))

# Joint Graphs

In [ ]:
fileNameType = "COMPARISON_ALL"

subtitles = "All positive patients v those that achieved a negative status"

titles = titles_list[1]


fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_JOINT.png", sep = "")



KM_FinalNegMainRule_all$plot <- KM_FinalNegMainRule_all$plot +

#change titles
labs(
    title = "A: Patients who achieved a negative status (2 negative swabs)", #title
     subtitle = NULL 
    
)  +

theme(text = element_text(size=20))


KM_FinalNegMainRule_all$table <- KM_FinalNegMainRule_all$table+

theme(text = element_text(size=20))




KM_all_all$plot <- KM_all_all$plot +

#change titles
labs(
    title = "B: All positive patients", #title
     subtitle = NULL 
    
)  +

theme(text = element_text(size=20))
 






splots <-list (  KM_FinalNegMainRule_all,
               KM_all_all
              )


p <- arrange_ggsurvplots(splots, 
                    print = TRUE,
                    ncol = 1, 
                    nrow = 2, 
                  #  title = paste(titles1,
                  #               " :", subtitles, 
                  #                " (",
                  #                titles,
                  #                ")",
                  #               sep = ""),
                    #surv.plot.height = 0.8,
                    risk.table.height = 0.1
                   
                   )




ggsave(file = fileName, 
       p,
      height = 30 , 
       width =  15)

p

In [ ]:
fileNameType = "COMPARISON_SEX"

subtitles = "Comparing stop rules"

titles = titles_list[2]


fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_JOINT.png", sep = "")


KM_all_sex$plot <- KM_all_sex$plot +

#change titles
labs(
    title = "A: All positive patients", #title
     subtitle = NULL 
    
)  
 

KM_FinalNegMainRule_sex$plot <- KM_FinalNegMainRule_sex$plot +

#change titles
labs(
    title = "B: Patients who achieved a negative status: Clinical rule", #title
     subtitle = NULL 
    
)  



KM_FinalNegNEWRule_sex$plot <- KM_FinalNegNEWRule_sex$plot +

#change titles
labs(
    title = "C: Patients who achieved a negative status: Alternate rule", #title
     subtitle = NULL 
    
)  




splots <-list (
    KM_all_sex,
               KM_FinalNegMainRule_sex,
              KM_FinalNegNEWRule_sex
              )


p <- arrange_ggsurvplots(splots, 
                    print = TRUE,
                    ncol = 3, 
                    nrow = 1, 
                    title = paste(titles1,
                                 " :", subtitles, 
                                  " (",
                                  titles,
                                  ")",
                                 sep = ""),
                    #surv.plot.height = 0.8,
                    risk.table.height = 0.2
                   
                   )




ggsave(file = fileName, 
       p,
      height = 10 , 
       width = 30 )

p

In [ ]:
fileNameType = "COMPARISON_AGE"

subtitles = "Comparing stop rules"

titles = titles_list[3]


fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_JOINT.png", sep = "")


KM_all_age$plot <- KM_all_age$plot +

#change titles
labs(
    title = "A: All positive patients", #title
     subtitle = NULL 
    
)  
 

KM_FinalNegMainRule_age$plot <- KM_FinalNegMainRule_age$plot +

#change titles
labs(
    title = "B: Patients who achieved a negative status: Clinical rule", #title
     subtitle = NULL 
    
)  



KM_FinalNegNEWRule_age$plot <- KM_FinalNegNEWRule_age$plot +

#change titles
labs(
    title = "C: Patients who achieved a negative status: Alternate rule", #title
     subtitle = NULL 
    
)  




splots <-list (
    KM_all_age,
               KM_FinalNegMainRule_age,
              KM_FinalNegNEWRule_age
              )


p <- arrange_ggsurvplots(splots, 
                    print = TRUE,
                    ncol = 3, 
                    nrow = 1, 
                    title = paste(titles1,
                                 " :", subtitles, 
                                  " (",
                                  titles,
                                  ")",
                                 sep = ""),
                    #surv.plot.height = 0.8,
                    risk.table.height = 0.2
                   
                   )




ggsave(file = fileName, 
       p,
      height = 10 , 
       width = 30 )

p

In [ ]:
fileNameType = "COMPARISON_AGESEX"

subtitles = "Comparing stop rules"

titles = titles_list[4]


fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_JOINT.png", sep = "")


KM_all_agesex$plot <- KM_all_agesex$plot +

#change titles
labs(
    title = "A: All positive patients", #title
     subtitle = NULL 
    
)  
 

KM_FinalNegMainRule_agesex$plot <- KM_FinalNegMainRule_agesex$plot +

#change titles
labs(
    title = "B: Patients who achieved a negative status: Clinical rule", #title
     subtitle = NULL 
    
)  



KM_FinalNegNEWRule_agesex$plot <- KM_FinalNegNEWRule_agesex$plot +

#change titles
labs(
    title = "C: Patients who achieved a negative status: Alternate rule", #title
     subtitle = NULL 
    
)  




splots <-list (KM_all_agesex,
               KM_FinalNegMainRule_agesex,
              KM_FinalNegNEWRule_agesex
              )


p <- arrange_ggsurvplots(splots, 
                    print = TRUE,
                    ncol = 3, 
                    nrow = 1, 
                    title = paste(titles1,
                                 " :", subtitles, 
                                  " (",
                                  titles,
                                  ")",
                                 sep = ""),
                    #surv.plot.height = 0.8,
                    risk.table.height = 0.2
                   
                   )




ggsave(file = fileName, 
       p,
      height = 10 , 
       width = 30 )

p

In [ ]:
fileNameType = "ALL_POS_AGE_SEX"

subtitles = "All positive patients"

titles = titles_list[2]


fileName <- paste("Outputs/Visuals/TTEAnalysis/", fileNameType,"_TTE_KMC_JOINT.png", sep = "")


KM_all_sex$plot <- KM_all_sex$plot +

#change titles
labs(
    title = "A: Sex", #title
     subtitle = NULL 
    
)  
 

KM_all_age$plot <- KM_all_age$plot +

#change titles
labs(
    title = "B: Age", #title
     subtitle = NULL 
    
)  




splots <-list (KM_all_sex,
               KM_all_age
              )


p <- arrange_ggsurvplots(splots, 
                    print = TRUE,
                    ncol = 2, 
                    nrow = 1, 
                    title = paste(titles1,
                                 " :", subtitles, 
                                  " (",
                                  titles,
                                  ")",
                                 sep = ""),
                    #surv.plot.height = 0.8,
                    risk.table.height = 0.2
                   
                   )




ggsave(file = fileName, 
       p,
      height = 10 , 
       width = 30 )

p

*************************
# **HEATMAPS**

REF: [Title and subtitle formatting](https://www.datanovia.com/en/blog/ggplot-title-subtitle-and-caption/#change-title-and-caption-style-font-size-color-and-face)

In [ ]:
captionsResult <- "Test result = overall test result for the day\nIf different test results from seperate sample, patient is assumed to be positive."

captionsStatus <- "A negative status is achieved after two consecutive negative test results more than 24 hours apart."

## 1. Censored after stop rule


In [ ]:
data <- SwabData_positive_3plus_censored

daysToNegStatus <- unique(select(data, 
                          Patient_ID, 
                          DaysToNegStatus,
                          #NumberOfTestsSinceFirstPOSITIVE
                         ))


outcome <- data$DaysToNegStatus



#bar_data <- unique(select(data, 
#                          Patient_ID, 
                          #DaysToNegStatus,
 #                         NumberOfTestsSinceFirstPOSITIVE
  #                       ))


#outcome_bar <- daysToNegStatus$DaysToNegStatus

fileNameType = "Censored"

titles <- "COVID19-Positive patients who had THREE or MORE swabs after their first positive"

captions <- "Patients have been censored after achiving their first negative status\n"

   
## 1.1 Stacked Bar Plot

In [ ]:
#create data frame of number of tests and number of neg testss
#number of tests

#tests since first positive
dataFirstPos <- data[data$DaySinceFirstPOSITIVETest >= 0, ]

testNum <- count(dataFirstPos$Patient_ID)

#change column names
names(testNum)[names(testNum) == "x"] <- "Patient_ID"
names(testNum)[names(testNum) == "freq"] <- "Total"
testNum[is.na(testNum)] <- 0


#number of negative tests
NEGtestNum <- count(dataFirstPos[dataFirstPos$Result_1 == "NO",]$Patient_ID)

#change column names
names(NEGtestNum)[names(NEGtestNum) == "x"] <- "Patient_ID"
names(NEGtestNum)[names(NEGtestNum) == "freq"] <- "With negative result"

#NEGtestNum[is.na(NEGtestNum)] <- 0



bar_data<- merge(x = testNum, y = NEGtestNum, by = "Patient_ID", all = TRUE)


bar_data<- merge(x = bar_data, y = daysToNegStatus, by = "Patient_ID", all = TRUE)

#replace NA with zeros
bar_data[is.na(bar_data)] <- 0





bar_data$Patient_ID <- factor(bar_data$Patient_ID,
                              levels= unique(bar_data$Patient_ID[order(
                                  
                                  -bar_data$DaysToNegStatus)]), #reverse order
                             ordered=TRUE)



#outcome_bar <- bar_data$DaysToNegStatus

#summary(bar_data)

In [ ]:
fileNameType1 <- "StackedBar" 

FileName <- paste("Outputs/Visuals/NumberTestsBar/PositivePatient_", fileNameType,"_", fileNameType1, ".png", sep = "")

#graph of number of tests



#reorder UR based on when the last test occured



num_tests_bar <-ggplot(data = bar_data) +

geom_bar( stat="identity",
            aes(x = bar_data$Patient_ID, 
                y = bar_data$Total, 
                fill = "Total",
              #  colour ="Total"
               )
        # alpha=0.3
        ) +

geom_bar(stat="identity",
         aes (x = bar_data$Patient_ID, 
              y = bar_data$"With negative result", 
              fill = "With negative result",
            # colour= "With negative result"
             ), 
        # alpha=0.3
        ) +

theme_bw() +

scale_fill_manual("Days of testing",
                  values=c("Total" = "grey", 
                             "With negative result" = "#fc8d62")) +


#flip x and y axis
coord_flip() +

#x axis label
labs(y = "No. days of testing first positive",
    x = element_blank()) +


#remove y ticks
theme(
    #axis.text.y = element_text(size=2),
    axis.text.y = element_blank(), #remove y labels
    axis.ticks.y = element_blank(), #remove y ticks
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7)
    
) + 


#fill based on number of tests




scale_y_reverse() + 


#legend
theme(legend.position = "bottom",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 


#remove background colour and grid
theme(panel.border = element_blank(), 
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank(), 
      panel.background = element_blank(),
     #line = element_blank()
       axis.line = element_line(colour = "black")
     ) +

scale_x_discrete(position = "top")



aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )




#num_tests_bar

## 1.2 Patient Test Result

### 1.2.1 Categorial heatmap only

In [ ]:
subtitles <- "Overall test result"

In [ ]:
#SWAB RESULT
fileNameType1 <- "TestResult" 

FileName <- paste("Outputs/Visuals/TestResults/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



#reorder UR based on when the last test occured
data$Patient_ID <- factor(data$Patient_ID,
                          levels= unique(data$Patient_ID[order(
                              -outcome)]), #reverse order
                          
                          ordered=TRUE)





#Plot the categorical data
heat_data  <- select (data,
                      Patient_ID,
                      DaySinceFirstPOSITIVETest,
                      
                      #variable for plotting
                      Result_1
                     )


#create colour pallet
cols = c("YES" = "#c51b7d",
         "NO" = "#4d9221",
         " " = "grey75")
                                                              
                                                              
#create the graph
heat_graph <- ggplot(
    
    #data
    heat_data, 
    
    #fill by variable and college
    aes(x = DaySinceFirstPOSITIVETest, 
        y = Patient_ID)) + 

#use categorical heatmap
geom_tile(aes(fill = Result_1),
           colour = "white") +

 
#change titles
labs(
    #title = "COVID19-Positive patients who had THREE or MORE swabs in total:", #title
    # subtitle = subtitles, 
    # caption = "Patients have been censored after achiving their first negative status", 
     
     x = "Day since first positive test",  #x axis
     y = "Patient") + #y axis 




theme(
    plot.title = element_text(color = "black", 
                              size = 12,
                              hjust = 0.5),
    plot.subtitle = element_text(color = "black",
                                 face = "bold",
                                 size = 12,
                                 hjust = 0.5),
    plot.caption = element_text(color = "grey", 
                                face = "italic",
                              hjust = 0),
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7),
    
    axis.title.y = element_text(size=8),
    
) +


#change colour of tiles
scale_fill_manual(name = "Test result",
                  values = cols,
                  
                  breaks = c("YES",
                              " ",
                              "NO"),
                  
                  labels = c("Positive",
                              "No result provided",
                              "Negative")) +






#legend
theme(legend.position = "bottom",
      legend.box="vertical",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 



theme(
    axis.text.y = element_text(size=2),
    #axis.text.y = element_blank(), #remove y labels
    #axis.ticks.y = element_blank() #remove y ticks
     ) + 


#add verticle line 10 days after day 0
geom_vline(xintercept = 10, 
           linetype="dotdash", 
           color = "grey21", 
           size=0.5) +


annotate("text", 
         label = "\n10 days after first positive test", 
         x = 10,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21") +



#add verticle line 14 days before day 0
geom_vline(xintercept = -14, 
           linetype="longdash", 
           color = "grey21", 
           size=0.5) +



annotate("text", 
         label = "14 days before first positive test\n", 
         x = -14,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21")










aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )


#heat_graph

### 1.2.2 Combined graph

In [ ]:
fileNameType1 <- "Combined" 

FileName <- paste("Outputs/Visuals/TestResults/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



combined <- ggarrange (num_tests_bar,
                       heat_graph,
                       nrow = 1,
                       ncol = 2,
                       widths = c(4, 6),
                       heights = c(20, 20)

            )


#ref: https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
combined<- annotate_figure(combined,
                           top = text_grob(paste(titles, ":\n", toupper(subtitles)), 
                                           color = "black", 
                                           face = "bold", 
                                           size = 10),
                           
                          
                            bottom = text_grob(paste(captions, captionsResult), 
                                               color = "grey",
                                               hjust = 0, 
                                               x = 0, 
                                               face = "italic", 
                                               size = 6)
                           )


aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       combined,
       height = 7 , 
       width = 7 )


#combined

## 1.3 Status


### 1.3.1 Categorial heatmap only

In [ ]:
subtitles <- "Clinical status"

In [ ]:
#SWAB RESULT
fileNameType1 <- "Status" 

FileName <- paste("Outputs/Visuals/Status/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")




#reorder UR based on when the last test occured
data$Patient_ID <- factor(data$Patient_ID,
                          levels= unique(data$Patient_ID[order(
                              -data$MaxDayTestSinceFirstPOSITIVETest)]), #reverse order
                          
                          ordered=TRUE)





#Plot the categorical data
heat_data  <- select (data,
                      Patient_ID,
                      DaySinceFirstPOSITIVETest,
                      
                      #variable for plotting
                      Status
                     )


#create colour pallet
cols = c("POS" = "#c51b7d",
         "NEG" = "#4d9221",
         " " = "grey75")
                                                              
                                                              
#create the graph
heat_graph <- ggplot(
    
    #data
    heat_data, 
    
    #fill by variable and college
    aes(x = DaySinceFirstPOSITIVETest, 
        y = Patient_ID)) + 

#use categorical heatmap
geom_tile(aes(fill = Status),
           colour = "white") +

 
#change titles
labs(
    #title = "COVID19-Positive patients who had THREE or MORE swabs in total:", #title
    # subtitle = subtitles, 
    # caption = "Patients have been censored after achiving their first negative status", 
     
     x = "Day since first positive test",  #x axis
     y = "Patient") + #y axis 


#legend
theme(legend.position = "bottom",
      legend.box="vertical",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 




theme(
    plot.title = element_text(color = "black", 
                              size = 12,
                              hjust = 0.5),
    plot.subtitle = element_text(color = "black",
                                 face = "bold",
                                 size = 12,
                                 hjust = 0.5),
    plot.caption = element_text(color = "grey", 
                                face = "italic",
                              hjust = 0),
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7),
    
    axis.title.y = element_text(size=8),
    
) +


#change colour of tiles
scale_fill_manual(name = "Status",
                  values = cols,
                  
                  breaks = c("POS",
                              " ",
                              "NEG"),
                  
                  labels = c("Positive",
                              "No result provided",
                              "Negative")) +


theme(#axis.text.y = element_text(size=2)
    axis.text.y = element_blank(), #remove y labels
    axis.ticks.y = element_blank() #remove y ticks
     ) + 


#add verticle line 10 days after day 0
#add verticle line 14 days before day 0
geom_vline(xintercept = 10, 
           linetype="dotdash", 
           color = "grey21", 
           size=0.5) +



annotate("text", 
         label = "\n10 days after first positive test", 
         x = 10,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21") +



#add verticle line 14 days before day 0
geom_vline(xintercept = -14, 
           linetype="longdash", 
           color = "grey21", 
           size=0.5) +



annotate("text", 
         label = "14 days before first positive test\n", 
         x = -14,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21")










aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )


#heat_graph

In [ ]:
fileNameType1 <- "Combined" 

FileName <- paste("Outputs/Visuals/Status/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



combined <- ggarrange (num_tests_bar,
                       heat_graph,
                       nrow = 1,
                       ncol = 2,
                       widths = c(4, 6),
                       heights = c(20)


            )


#ref: https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
combined<- annotate_figure(combined,
                           top = text_grob(paste(titles, ":\n", toupper(subtitles)), 
                                           color = "black", 
                                           face = "bold", 
                                           size = 10),
                           
                          
                            bottom = text_grob(paste(captions,captionsStatus), 
                                               color = "grey",
                                               hjust = 0, 
                                               x = 0, 
                                               face = "italic", 
                                               size = 6)
                           )



aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       combined,
       height = 7 , 
       width = 7 )



*****************


## 2. UNcensored after stop rule


In [ ]:
data <- SwabData_positive_3plus

daysToNegStatus <- unique(select(data, 
                          Patient_ID, 
                          DaysToNegStatus_sensitiv,
                          #NumberOfTestsSinceFirstPOSITIVE
                         ))


outcome <- data$DaysToNegStatus_sensitiv



#bar_data <- unique(select(data, 
#                          Patient_ID, 
                          #DaysToNegStatus,
 #                         NumberOfTestsSinceFirstPOSITIVE
  #                       ))


#outcome_bar <- daysToNegStatus$DaysToNegStatus

fileNameType = "Uncensored"

titles <- "COVID19-Positive patients who had THREE or MORE swabs after their first positive"

captions <- "Patients have NOT been censored after achiving their first negative status"

In [ ]:
max(SwabData_positive_3plus$DaysToNegStatus)

   
## 1.1 Stacked Bar Plot

In [ ]:
#create data frame of number of tests and number of neg testss
#number of tests

#tests since first positive
dataFirstPos <- data[data$DaySinceFirstPOSITIVETest >= 0, ]

testNum <- count(dataFirstPos$Patient_ID)

#change column names
names(testNum)[names(testNum) == "x"] <- "Patient_ID"
names(testNum)[names(testNum) == "freq"] <- "Total"
testNum[is.na(testNum)] <- 0


#number of negative tests
NEGtestNum <- count(dataFirstPos[dataFirstPos$Result_1 == "NO",]$Patient_ID)

#change column names
names(NEGtestNum)[names(NEGtestNum) == "x"] <- "Patient_ID"
names(NEGtestNum)[names(NEGtestNum) == "freq"] <- "With negative result"

#NEGtestNum[is.na(NEGtestNum)] <- 0



bar_data<- merge(x = testNum, y = NEGtestNum, by = "Patient_ID", all = TRUE)


bar_data<- merge(x = bar_data, y = daysToNegStatus, by = "Patient_ID", all = TRUE)

#replace NA with zeros
bar_data[is.na(bar_data)] <- 0





bar_data$Patient_ID <- factor(bar_data$Patient_ID,
                              levels= unique(bar_data$Patient_ID[order(
                                  
                                  -bar_data$DaysToNegStatus)]), #reverse order
                             ordered=TRUE)



#outcome_bar <- bar_data$DaysToNegStatus

#summary(bar_data)

In [ ]:
fileNameType1 <- "StackedBar" 

FileName <- paste("Outputs/Visuals/NumberTestsBar/PositivePatient_", fileNameType,"_", fileNameType1, ".png", sep = "")

#graph of number of tests



#reorder UR based on when the last test occured



num_tests_bar <-ggplot(data = bar_data) +

geom_bar( stat="identity",
            aes(x = bar_data$Patient_ID, 
                y = bar_data$Total, 
                fill = "Total",
              #  colour ="Total"
               )
        # alpha=0.3
        ) +

geom_bar(stat="identity",
         aes (x = bar_data$Patient_ID, 
              y = bar_data$"With negative result", 
              fill = "With negative result",
            # colour= "With negative result"
             ), 
        # alpha=0.3
        ) +

theme_bw() +

scale_fill_manual("Days of testing",
                  values=c("Total" = "grey", 
                             "With negative result" = "#fc8d62")) +


#flip x and y axis
coord_flip() +

#x axis label
labs(y = "No. days of testing first positive",
    x = element_blank()) +


#remove y ticks
theme(
    #axis.text.y = element_text(size=2),
    axis.text.y = element_blank(), #remove y labels
    axis.ticks.y = element_blank(), #remove y ticks
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7)
    
) + 


#fill based on number of tests




scale_y_reverse() + 


#legend
theme(legend.position = "bottom",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 


#remove background colour and grid
theme(panel.border = element_blank(), 
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank(), 
      panel.background = element_blank(),
     #line = element_blank()
       axis.line = element_line(colour = "black")
     ) +

scale_x_discrete(position = "top")



aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )




#num_tests_bar

## 1.2 Patient Test Result

### 1.2.1 Categorial heatmap only

In [ ]:
subtitles <- "Overall test result"

In [ ]:
#SWAB RESULT
fileNameType1 <- "TestResult" 

FileName <- paste("Outputs/Visuals/TestResults/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



#reorder UR based on when the last test occured
data$Patient_ID <- factor(data$Patient_ID,
                          levels= unique(data$Patient_ID[order(
                              -outcome)]), #reverse order
                          
                          ordered=TRUE)





#Plot the categorical data
heat_data  <- select (data,
                      Patient_ID,
                      DaySinceFirstPOSITIVETest,
                      
                      #variable for plotting
                      Result_1
                     )


#create colour pallet
cols = c("YES" = "#c51b7d",
         "NO" = "#4d9221",
         " " = "grey75")
                                                              
                                                              
#create the graph
heat_graph <- ggplot(
    
    #data
    heat_data, 
    
    #fill by variable and college
    aes(x = DaySinceFirstPOSITIVETest, 
        y = Patient_ID)) + 

#use categorical heatmap
geom_tile(aes(fill = Result_1),
           colour = "white") +

 
#change titles
labs(
    #title = "COVID19-Positive patients who had THREE or MORE swabs in total:", #title
    # subtitle = subtitles, 
    # caption = "Patients have been censored after achiving their first negative status", 
     
     x = "Day since first positive test",  #x axis
     y = "Patient") + #y axis 




theme(
    plot.title = element_text(color = "black", 
                              size = 12,
                              hjust = 0.5),
    plot.subtitle = element_text(color = "black",
                                 face = "bold",
                                 size = 12,
                                 hjust = 0.5),
    plot.caption = element_text(color = "grey", 
                                face = "italic",
                              hjust = 0),
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7),
    
    axis.title.y = element_text(size=8),
    
) +


#change colour of tiles
scale_fill_manual(name = "Test result",
                  values = cols,
                  
                  breaks = c("YES",
                              " ",
                              "NO"),
                  
                  labels = c("Positive",
                              "No result provided",
                              "Negative")) +






#legend
theme(legend.position = "bottom",
      legend.box="vertical",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 



theme(
    #axis.text.y = element_text(size=2),
    axis.text.y = element_blank(), #remove y labels
    axis.ticks.y = element_blank() #remove y ticks
     ) + 


#add verticle line 10 days after day 0
geom_vline(xintercept = 10, 
           linetype="dotdash", 
           color = "grey21", 
           size=0.5) +


annotate("text", 
         label = "\n10 days after first positive test", 
         x = 10,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21") +



#add verticle line 14 days before day 0
geom_vline(xintercept = -14, 
           linetype="longdash", 
           color = "grey21", 
           size=0.5) +



annotate("text", 
         label = "14 days before first positive test\n", 
         x = -14,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21")










aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )


#heat_graph

### 1.2.2 Combined graph

In [ ]:
fileNameType1 <- "Combined" 

FileName <- paste("Outputs/Visuals/TestResults/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



combined <- ggarrange (num_tests_bar,
                       heat_graph,
                       nrow = 1,
                       ncol = 2,
                       widths = c(4, 6),
                       heights = c(20, 20)

            )


#ref: https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
combined<- annotate_figure(combined,
                           top = text_grob(paste(titles, ":\n", toupper(subtitles)), 
                                           color = "black", 
                                           face = "bold", 
                                           size = 10),
                           
                          
                            bottom = text_grob(paste(captions, captionsResult), 
                                               color = "grey",
                                               hjust = 0, 
                                               x = 0, 
                                               face = "italic", 
                                               size = 6)
                           )


aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       combined,
       height = 7 , 
       width = 7 )


#combined

## 1.3 Status


### 1.3.1 Categorial heatmap only

In [ ]:
subtitles <- "Clinical status"

In [ ]:
#SWAB RESULT
fileNameType1 <- "Status" 

FileName <- paste("Outputs/Visuals/Status/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")


data <- data[data$DaySinceFirstPOSITIVETest >=0, ]

#reorder UR based on when the last test occured
data$Patient_ID <- factor(data$Patient_ID,
                          levels= unique(data$Patient_ID[order(
                              -data$MaxDayTestSinceFirstPOSITIVETest)]), #reverse order
                          
                          ordered=TRUE)





#Plot the categorical data
heat_data  <- select (data,
                      Patient_ID,
                      DaySinceFirstPOSITIVETest,
                      
                      #variable for plotting
                      Status
                     )


#create colour pallet
cols = c("POS" = "#c51b7d",
         "NEG" = "#4d9221",
         " " = "grey75")
                                                              
                                                              
#create the graph
heat_graph <- ggplot(
    
    #data
    heat_data, 
    
    #fill by variable and college
    aes(x = DaySinceFirstPOSITIVETest, 
        y = Patient_ID)) + 

#use categorical heatmap
geom_tile(aes(fill = Status),
           colour = "white") +

 
#change titles
labs(
    #title = "COVID19-Positive patients who had THREE or MORE swabs in total:", #title
    # subtitle = subtitles, 
    # caption = "Patients have been censored after achiving their first negative status", 
     
     x = "Day since first positive test",  #x axis
     y = "Patient") + #y axis 


#legend
theme(legend.position = "bottom",
      legend.box="vertical",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 




theme(
    plot.title = element_text(color = "black", 
                              size = 12,
                              hjust = 0.5),
    plot.subtitle = element_text(color = "black",
                                 face = "bold",
                                 size = 12,
                                 hjust = 0.5),
    plot.caption = element_text(color = "grey", 
                                face = "italic",
                              hjust = 0),
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7),
    
    axis.title.y = element_text(size=8),
    
) +


#change colour of tiles
scale_fill_manual(name = "Status",
                  values = cols,
                  
                  breaks = c("POS",
                              " ",
                              "NEG"),
                  
                  labels = c("Positive",
                              "No result provided",
                              "Negative")) +


theme(#axis.text.y = element_text(size=2)
    axis.text.y = element_blank(), #remove y labels
    axis.ticks.y = element_blank() #remove y ticks
     ) + 


#add verticle line 10 days after day 0
#add verticle line 14 days before day 0
geom_vline(xintercept = 10, 
           linetype="dotdash", 
           color = "grey21", 
           size=0.5) +



annotate("text", 
         label = "\n10 days after first positive test", 
         x = 10,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21") 
#+



#add verticle line 14 days before day 0
#geom_vline(xintercept = -14, 
 #          linetype="longdash", 
  #         color = "grey21", 
   #        size=0.5) +



#annotate("text", 
 #        label = "14 days before first positive test\n", 
  #       x = -14,
   #      y = length(unique(data$Patient_ID))*0.90, 
    #     size = 2, 
     #    angle = 90, 
      #   colour = "grey21")










aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )

print(FileName)
heat_graph

In [ ]:
fileNameType1 <- "Combined" 

FileName <- paste("Outputs/Visuals/Status/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



combined <- ggarrange (num_tests_bar,
                       heat_graph,
                       nrow = 1,
                       ncol = 2,
                       widths = c(4, 6),
                       heights = c(20)


            )


#ref: https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
combined<- annotate_figure(combined,
                           top = text_grob(paste(titles, ":\n", toupper(subtitles)), 
                                           color = "black", 
                                           face = "bold", 
                                           size = 10),
                           
                          
                            bottom = text_grob(paste(captions,captionsStatus), 
                                               color = "grey",
                                               hjust = 0, 
                                               x = 0, 
                                               face = "italic", 
                                               size = 6)
                           )



aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       combined,
       height = 7 , 
       width = 7 )



*****************


## 3. Patients that changed status POS -- NEG -- POS


In [ ]:
data <- SwabData_status_Change

daysToNegStatus <- unique(select(data, 
                          Patient_ID, 
                          DaysToNegStatus_sensitiv,
                          #NumberOfTestsSinceFirstPOSITIVE
                         ))


outcome <- data$DaysToNegStatus_sensitiv



#bar_data <- unique(select(data, 
#                          Patient_ID, 
                          #DaysToNegStatus,
 #                         NumberOfTestsSinceFirstPOSITIVE
  #                       ))


#outcome_bar <- daysToNegStatus$DaysToNegStatus

fileNameType = "StatusChange"

titles <- "COVID19-Positive patients who changed back to a positive status after achiving a negative status"

captions <- "Patients have NOT been censored after achiving their first negative status"

   
## 1.1 Stacked Bar Plot

In [ ]:
#create data frame of number of tests and number of neg testss
#number of tests

#tests since first positive
dataFirstPos <- data[data$DaySinceFirstPOSITIVETest >= 0, ]

testNum <- count(dataFirstPos$Patient_ID)

#change column names
names(testNum)[names(testNum) == "x"] <- "Patient_ID"
names(testNum)[names(testNum) == "freq"] <- "Total"
testNum[is.na(testNum)] <- 0


#number of negative tests
NEGtestNum <- count(dataFirstPos[dataFirstPos$Result_1 == "NO",]$Patient_ID)

#change column names
names(NEGtestNum)[names(NEGtestNum) == "x"] <- "Patient_ID"
names(NEGtestNum)[names(NEGtestNum) == "freq"] <- "With negative result"

#NEGtestNum[is.na(NEGtestNum)] <- 0



bar_data<- merge(x = testNum, y = NEGtestNum, by = "Patient_ID", all = TRUE)


bar_data<- merge(x = bar_data, y = daysToNegStatus, by = "Patient_ID", all = TRUE)

#replace NA with zeros
bar_data[is.na(bar_data)] <- 0





bar_data$Patient_ID <- factor(bar_data$Patient_ID,
                              levels= unique(bar_data$Patient_ID[order(
                                  
                                  -bar_data$DaysToNegStatus)]), #reverse order
                             ordered=TRUE)



#outcome_bar <- bar_data$DaysToNegStatus

#summary(bar_data)

In [ ]:
fileNameType1 <- "StackedBar" 

FileName <- paste("Outputs/Visuals/NumberTestsBar/PositivePatient_", fileNameType,"_", fileNameType1, ".png", sep = "")

#graph of number of tests



#reorder UR based on when the last test occured



num_tests_bar <-ggplot(data = bar_data) +

geom_bar( stat="identity",
            aes(x = bar_data$Patient_ID, 
                y = bar_data$Total, 
                fill = "Total",
              #  colour ="Total"
               )
        # alpha=0.3
        ) +

geom_bar(stat="identity",
         aes (x = bar_data$Patient_ID, 
              y = bar_data$"With negative result", 
              fill = "With negative result",
            # colour= "With negative result"
             ), 
        # alpha=0.3
        ) +

theme_bw() +

scale_fill_manual("Days of testing",
                  values=c("Total" = "grey", 
                             "With negative result" = "#fc8d62")) +


#flip x and y axis
coord_flip() +

#x axis label
labs(y = "No. days tested after first positive",
    x = element_blank()) +


#remove y ticks
theme(
    #axis.text.y = element_text(size=2),
    axis.text.y = element_blank(), #remove y labels
    axis.ticks.y = element_blank(), #remove y ticks
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7)
    
) + 


#fill based on number of tests




scale_y_reverse() + 


#legend
theme(legend.position = "bottom",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 


#remove background colour and grid
theme(panel.border = element_blank(), 
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank(), 
      panel.background = element_blank(),
     #line = element_blank()
       axis.line = element_line(colour = "black")
     ) +

scale_x_discrete(position = "top")



aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )




#num_tests_bar

## 1.2 Patient Test Result

### 1.2.1 Categorial heatmap only

In [ ]:
subtitles <- "Overall test result"

In [ ]:
#SWAB RESULT
fileNameType1 <- "TestResult" 

FileName <- paste("Outputs/Visuals/TestResults/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



#reorder UR based on when the last test occured
data$Patient_ID <- factor(data$Patient_ID,
                          levels= unique(data$Patient_ID[order(
                              -outcome)]), #reverse order
                          
                          ordered=TRUE)





#Plot the categorical data
heat_data  <- select (data,
                      Patient_ID,
                      DaySinceFirstPOSITIVETest,
                      
                      #variable for plotting
                      Result_1
                     )


#create colour pallet
cols = c("YES" = "#c51b7d",
         "NO" = "#4d9221",
         " " = "grey75")
                                                              
                                                              
#create the graph
heat_graph <- ggplot(
    
    #data
    heat_data, 
    
    #fill by variable and college
    aes(x = DaySinceFirstPOSITIVETest, 
        y = Patient_ID)) + 

#use categorical heatmap
geom_tile(aes(fill = Result_1),
           colour = "white") +

 
#change titles
labs(
    #title = "COVID19-Positive patients who had THREE or MORE swabs in total:", #title
    # subtitle = subtitles, 
    # caption = "Patients have been censored after achiving their first negative status", 
     
     x = "Day since first positive test",  #x axis
     y = "Patient") + #y axis 




theme(
    plot.title = element_text(color = "black", 
                              size = 12,
                              hjust = 0.5),
    plot.subtitle = element_text(color = "black",
                                 face = "bold",
                                 size = 12,
                                 hjust = 0.5),
    plot.caption = element_text(color = "grey", 
                                face = "italic",
                              hjust = 0),
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7),
    
    axis.title.y = element_text(size=8),
    
) +


#change colour of tiles
scale_fill_manual(name = "Test result",
                  values = cols,
                  
                  breaks = c("YES",
                              " ",
                              "NO"),
                  
                  labels = c("Positive",
                              "No result provided",
                              "Negative")) +






#legend
theme(legend.position = "bottom",
      legend.box="vertical",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 



theme(
    axis.text.y = element_text(size=2),
    #axis.text.y = element_blank(), #remove y labels
    #axis.ticks.y = element_blank() #remove y ticks
     ) + 


#add verticle line 10 days after day 0
geom_vline(xintercept = 10, 
           linetype="dotdash", 
           color = "grey21", 
           size=0.5) +


annotate("text", 
         label = "\n10 days after first positive test", 
         x = 10,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21") +



#add verticle line 14 days before day 0
geom_vline(xintercept = -14, 
           linetype="longdash", 
           color = "grey21", 
           size=0.5) +



annotate("text", 
         label = "14 days before first positive test\n", 
         x = -14,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21")










aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )


#heat_graph

### 1.2.2 Combined graph

In [ ]:
fileNameType1 <- "Combined" 

FileName <- paste("Outputs/Visuals/TestResults/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



combined <- ggarrange (num_tests_bar,
                       heat_graph,
                       nrow = 1,
                       ncol = 2,
                       widths = c(4, 6),
                       heights = c(20, 20)

            )


#ref: https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
combined<- annotate_figure(combined,
                           top = text_grob(paste(titles, ":\n", toupper(subtitles)), 
                                           color = "black", 
                                           face = "bold", 
                                           size = 10),
                           
                          
                            bottom = text_grob(paste(captions, captionsResult), 
                                               color = "grey",
                                               hjust = 0, 
                                               x = 0, 
                                               face = "italic", 
                                               size = 6)
                           )


aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       combined,
       height = 7 , 
       width = 7 )


#combined

## 1.3 Status


### 1.3.1 Categorial heatmap only

In [ ]:
subtitles <- "Clinical status"

In [ ]:
#SWAB RESULT
fileNameType1 <- "Status" 

FileName <- paste("Outputs/Visuals/Status/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")




#reorder UR based on when the last test occured
data$Patient_ID <- factor(data$Patient_ID,
                          levels= unique(data$Patient_ID[order(
                              -data$MaxDayTestSinceFirstPOSITIVETest)]), #reverse order
                          
                          ordered=TRUE)





#Plot the categorical data
heat_data  <- select (data,
                      Patient_ID,
                      DaySinceFirstPOSITIVETest,
                      
                      #variable for plotting
                      Status
                     )


#create colour pallet
cols = c("POS" = "#c51b7d",
         "NEG" = "#4d9221",
         " " = "grey75")
                                                              
                                                              
#create the graph
heat_graph <- ggplot(
    
    #data
    heat_data, 
    
    #fill by variable and college
    aes(x = DaySinceFirstPOSITIVETest, 
        y = Patient_ID)) + 

#use categorical heatmap
geom_tile(aes(fill = Status),
           colour = "white") +

 
#change titles
labs(
    #title = "COVID19-Positive patients who had THREE or MORE swabs in total:", #title
    # subtitle = subtitles, 
    # caption = "Patients have been censored after achiving their first negative status", 
     
     x = "Day since first positive test",  #x axis
     y = "Patient") + #y axis 


#legend
theme(legend.position = "bottom",
      legend.box="vertical",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 




theme(
    plot.title = element_text(color = "black", 
                              size = 12,
                              hjust = 0.5),
    plot.subtitle = element_text(color = "black",
                                 face = "bold",
                                 size = 12,
                                 hjust = 0.5),
    plot.caption = element_text(color = "grey", 
                                face = "italic",
                              hjust = 0),
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7),
    
    axis.title.y = element_text(size=8),
    
) +


#change colour of tiles
scale_fill_manual(name = "Status",
                  values = cols,
                  
                  breaks = c("POS",
                              " ",
                              "NEG"),
                  
                  labels = c("Positive",
                              "No result provided",
                              "Negative")) +


theme(#axis.text.y = element_text(size=2),
    axis.text.y = element_blank(), #remove y labels
    axis.ticks.y = element_blank() #remove y ticks
     ) + 


#add verticle line 10 days after day 0
#add verticle line 14 days before day 0
geom_vline(xintercept = 10, 
           linetype="dotdash", 
           color = "grey21", 
           size=0.5) +



annotate("text", 
         label = "\n10 days after first positive test", 
         x = 10,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21") +



#add verticle line 14 days before day 0
geom_vline(xintercept = -14, 
           linetype="longdash", 
           color = "grey21", 
           size=0.5) +



annotate("text", 
         label = "14 days before first positive test\n", 
         x = -14,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21")










aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )


#heat_graph

In [ ]:
fileNameType1 <- "Combined" 

FileName <- paste("Outputs/Visuals/Status/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



combined <- ggarrange (num_tests_bar,
                       heat_graph,
                       nrow = 1,
                       ncol = 2,
                       widths = c(4, 6),
                       heights = c(20)


            )


#ref: https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
combined<- annotate_figure(combined,
                           top = text_grob(paste(titles, ":\n", toupper(subtitles)), 
                                           color = "black", 
                                           face = "bold", 
                                           size = 10),
                           
                          
                            bottom = text_grob(paste(captions,captionsStatus), 
                                               color = "grey",
                                               hjust = 0, 
                                               x = 0, 
                                               face = "italic", 
                                               size = 6)
                           )



aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       combined,
       height = 7 , 
       width = 7 )



*****************


## 4. ALL Patients with tests PRIOR to first positive, irrespective of number of tests after first positive


In [ ]:
data <- SwabData_prior_test

daysToNegStatus <- unique(select(data, 
                          Patient_ID, 
                          DaysToNegStatus_sensitiv,
                          #NumberOfTestsSinceFirstPOSITIVE
                         ))


outcome <- data$DaysToNegStatus_sensitiv



#bar_data <- unique(select(data, 
#                          Patient_ID, 
                          #DaysToNegStatus,
 #                         NumberOfTestsSinceFirstPOSITIVE
  #                       ))


#outcome_bar <- daysToNegStatus$DaysToNegStatus

fileNameType = "NegPrior"

titles <- "COVID19-Positive patients who changed back to a positive status after achiving a negative status"

captions <- "Patients have NOT been censored after achiving their first negative status"

   
## 1.1 Stacked Bar Plot

In [ ]:
#create data frame of number of tests and number of neg testss
#number of tests

#tests since first positive
dataFirstPos <- data[data$DaySinceFirstPOSITIVETest >= 0, ]

testNum <- count(dataFirstPos$Patient_ID)

#change column names
names(testNum)[names(testNum) == "x"] <- "Patient_ID"
names(testNum)[names(testNum) == "freq"] <- "Total"
testNum[is.na(testNum)] <- 0


#number of negative tests
NEGtestNum <- count(dataFirstPos[dataFirstPos$Result_1 == "NO",]$Patient_ID)

#change column names
names(NEGtestNum)[names(NEGtestNum) == "x"] <- "Patient_ID"
names(NEGtestNum)[names(NEGtestNum) == "freq"] <- "With negative result"

#NEGtestNum[is.na(NEGtestNum)] <- 0



bar_data<- merge(x = testNum, y = NEGtestNum, by = "Patient_ID", all = TRUE)


bar_data<- merge(x = bar_data, y = daysToNegStatus, by = "Patient_ID", all = TRUE)

#replace NA with zeros
bar_data[is.na(bar_data)] <- 0





bar_data$Patient_ID <- factor(bar_data$Patient_ID,
                              levels= unique(bar_data$Patient_ID[order(
                                  
                                  -bar_data$DaysToNegStatus)]), #reverse order
                             ordered=TRUE)



#outcome_bar <- bar_data$DaysToNegStatus

#summary(bar_data)

In [ ]:
fileNameType1 <- "StackedBar" 

FileName <- paste("Outputs/Visuals/NumberTestsBar/PositivePatient_", fileNameType,"_", fileNameType1, ".png", sep = "")

#graph of number of tests



#reorder UR based on when the last test occured



num_tests_bar <-ggplot(data = bar_data) +

geom_bar( stat="identity",
            aes(x = bar_data$Patient_ID, 
                y = bar_data$Total, 
                fill = "Total",
              #  colour ="Total"
               )
        # alpha=0.3
        ) +

geom_bar(stat="identity",
         aes (x = bar_data$Patient_ID, 
              y = bar_data$"With negative result", 
              fill = "With negative result",
            # colour= "With negative result"
             ), 
        # alpha=0.3
        ) +

theme_bw() +

scale_fill_manual("Days of testing",
                  values=c("Total" = "grey", 
                             "With negative result" = "#fc8d62")) +


#flip x and y axis
coord_flip() +

#x axis label
labs(y = "No. days tested after first positive",
    x = element_blank()) +


#remove y ticks
theme(
    #axis.text.y = element_text(size=2),
    axis.text.y = element_blank(), #remove y labels
    axis.ticks.y = element_blank(), #remove y ticks
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7)
    
) + 


#fill based on number of tests




scale_y_reverse() + 


#legend
theme(legend.position = "bottom",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 


#remove background colour and grid
theme(panel.border = element_blank(), 
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank(), 
      panel.background = element_blank(),
     #line = element_blank()
       axis.line = element_line(colour = "black")
     ) +

scale_x_discrete(position = "top")



aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )




#num_tests_bar

## 1.2 Patient Test Result

### 1.2.1 Categorial heatmap only

In [ ]:
subtitles <- "Overall test result"

In [ ]:
#SWAB RESULT
fileNameType1 <- "TestResult" 

FileName <- paste("Outputs/Visuals/TestResults/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



#reorder UR based on when the last test occured
data$Patient_ID <- factor(data$Patient_ID,
                          levels= unique(data$Patient_ID[order(
                              -outcome)]), #reverse order
                          
                          ordered=TRUE)





#Plot the categorical data
heat_data  <- select (data,
                      Patient_ID,
                      DaySinceFirstPOSITIVETest,
                      
                      #variable for plotting
                      Result_1
                     )


#create colour pallet
cols = c("YES" = "#c51b7d",
         "NO" = "#4d9221",
         " " = "grey75")
                                                              
                                                              
#create the graph
heat_graph <- ggplot(
    
    #data
    heat_data, 
    
    #fill by variable and college
    aes(x = DaySinceFirstPOSITIVETest, 
        y = Patient_ID)) + 

#use categorical heatmap
geom_tile(aes(fill = Result_1),
           colour = "white") +

 
#change titles
labs(
    #title = "COVID19-Positive patients who had THREE or MORE swabs in total:", #title
    # subtitle = subtitles, 
    # caption = "Patients have been censored after achiving their first negative status", 
     
     x = "Day since first positive test",  #x axis
     y = "Patient") + #y axis 




theme(
    plot.title = element_text(color = "black", 
                              size = 12,
                              hjust = 0.5),
    plot.subtitle = element_text(color = "black",
                                 face = "bold",
                                 size = 12,
                                 hjust = 0.5),
    plot.caption = element_text(color = "grey", 
                                face = "italic",
                              hjust = 0),
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7),
    
    axis.title.y = element_text(size=8),
    
) +


#change colour of tiles
scale_fill_manual(name = "Test result",
                  values = cols,
                  
                  breaks = c("YES",
                              " ",
                              "NO"),
                  
                  labels = c("Positive",
                              "No result provided",
                              "Negative")) +






#legend
theme(legend.position = "bottom",
      legend.box="vertical",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 



theme(
    #axis.text.y = element_text(size=2),
    axis.text.y = element_blank(), #remove y labels
    axis.ticks.y = element_blank() #remove y ticks
     ) + 


#add verticle line 10 days after day 0
geom_vline(xintercept = 10, 
           linetype="dotdash", 
           color = "grey21", 
           size=0.5) +


annotate("text", 
         label = "\n10 days after first positive test", 
         x = 10,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21") +



#add verticle line 14 days before day 0
geom_vline(xintercept = -14, 
           linetype="longdash", 
           color = "grey21", 
           size=0.5) +



annotate("text", 
         label = "14 days before first positive test\n", 
         x = -14,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21")










aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )


#heat_graph

### 1.2.2 Combined graph

In [ ]:
fileNameType1 <- "Combined" 

FileName <- paste("Outputs/Visuals/TestResults/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



combined <- ggarrange (num_tests_bar,
                       heat_graph,
                       nrow = 1,
                       ncol = 2,
                       widths = c(4, 6),
                       heights = c(20, 20)

            )


#ref: https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
combined<- annotate_figure(combined,
                           top = text_grob(paste(titles, ":\n", toupper(subtitles)), 
                                           color = "black", 
                                           face = "bold", 
                                           size = 10),
                           
                          
                            bottom = text_grob(paste(captions, captionsResult), 
                                               color = "grey",
                                               hjust = 0, 
                                               x = 0, 
                                               face = "italic", 
                                               size = 6)
                           )


aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       combined,
       height = 7 , 
       width = 7 )


#combined

## 1.3 Status


### 1.3.1 Categorial heatmap only

In [ ]:
subtitles <- "Clinical status"

In [ ]:
#SWAB RESULT
fileNameType1 <- "Status" 

FileName <- paste("Outputs/Visuals/Status/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")




#reorder UR based on when the last test occured
data$Patient_ID <- factor(data$Patient_ID,
                          levels= unique(data$Patient_ID[order(
                              -data$MaxDayTestSinceFirstPOSITIVETest)]), #reverse order
                          
                          ordered=TRUE)





#Plot the categorical data
heat_data  <- select (data,
                      Patient_ID,
                      DaySinceFirstPOSITIVETest,
                      
                      #variable for plotting
                      Status
                     )


#create colour pallet
cols = c("POS" = "#c51b7d",
         "NEG" = "#4d9221",
         " " = "grey75")
                                                              
                                                              
#create the graph
heat_graph <- ggplot(
    
    #data
    heat_data, 
    
    #fill by variable and college
    aes(x = DaySinceFirstPOSITIVETest, 
        y = Patient_ID)) + 

#use categorical heatmap
geom_tile(aes(fill = Status),
           colour = "white") +

 
#change titles
labs(
    #title = "COVID19-Positive patients who had THREE or MORE swabs in total:", #title
    # subtitle = subtitles, 
    # caption = "Patients have been censored after achiving their first negative status", 
     
     x = "Day since first positive test",  #x axis
     y = "Patient") + #y axis 


#legend
theme(legend.position = "bottom",
      legend.box="vertical",
      legend.title=element_text(size=7), 
      legend.text=element_text(size=7)) + 




theme(
    plot.title = element_text(color = "black", 
                              size = 12,
                              hjust = 0.5),
    plot.subtitle = element_text(color = "black",
                                 face = "bold",
                                 size = 12,
                                 hjust = 0.5),
    plot.caption = element_text(color = "grey", 
                                face = "italic",
                              hjust = 0),
    
    axis.title.x = element_text(size=8),
    axis.text.x = element_text(size=7),
    
    axis.title.y = element_text(size=8),
    
) +


#change colour of tiles
scale_fill_manual(name = "Status",
                  values = cols,
                  
                  breaks = c("POS",
                              " ",
                              "NEG"),
                  
                  labels = c("Positive",
                              "No result provided",
                              "Negative")) +


theme(#axis.text.y = element_text(size=2)
    axis.text.y = element_blank(), #remove y labels
    axis.ticks.y = element_blank() #remove y ticks
     ) + 


#add verticle line 10 days after day 0
#add verticle line 14 days before day 0
geom_vline(xintercept = 10, 
           linetype="dotdash", 
           color = "grey21", 
           size=0.5) +



annotate("text", 
         label = "\n10 days after first positive test", 
         x = 10,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21") +



#add verticle line 14 days before day 0
geom_vline(xintercept = -14, 
           linetype="longdash", 
           color = "grey21", 
           size=0.5) +



annotate("text", 
         label = "14 days before first positive test\n", 
         x = -14,
         y = length(unique(data$Patient_ID))*0.90, 
         size = 2, 
         angle = 90, 
         colour = "grey21")










aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       height = 7 , 
       width = 7 )


#heat_graph

In [ ]:
fileNameType1 <- "Combined" 

FileName <- paste("Outputs/Visuals/Status/PositivePatient_", fileNameType,"_",subtitles, "_", fileNameType1, ".png", sep = "")



combined <- ggarrange (num_tests_bar,
                       heat_graph,
                       nrow = 1,
                       ncol = 2,
                       widths = c(4, 6),
                       heights = c(20)


            )


#ref: https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
combined<- annotate_figure(combined,
                           top = text_grob(paste(titles, ":\n", toupper(subtitles)), 
                                           color = "black", 
                                           face = "bold", 
                                           size = 10),
                           
                          
                            bottom = text_grob(paste(captions,captionsStatus), 
                                               color = "grey",
                                               hjust = 0, 
                                               x = 0, 
                                               face = "italic", 
                                               size = 6)
                           )



aspect_ratio <- 5

height <- 7

ggsave(FileName, 
       combined,
       height = 7 , 
       width = 7 )

